In [1]:
import pandas as pd
import glob as glob
import numpy as np
import seaborn as sns
import matplotlib as plt
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth', None)


In [2]:
### bring in 6Sense

sense6 = pd.read_csv('6Sense Web Data.csv')
sense6['url'] = sense6['url'].astype('str').str.lower()
sense6['domain'] = sense6['domain'].astype('str').str.lower()
sense6.sample(5)

,name,country,domain,url,visiteddate,DateRange
4348638,CAMPARI,Italy,campari.com,https://www.sailpoint.com/identity-library/identity-and-access-management/,2020-03-25,last14days
2942337,THRIVENTFINANCIALFORLUTHERANSFOUNDATION,United States,thrivent.com,https://www.content.sailpoint.com/l/explore-ai-and-machine-learning?elqct=website&elqchannel=organicdirect&elqcta=solutions-ai-machine-learning&elqcname=19-q4-sps-ww-website-pathfactoryweb&sls=19-q4-sps-ww-website-pathfactoryweb&elqcampaignid=2628,2019-12-04,last14days
3599194,RBC,Canada,rbc.com,https://www.sailpoint.com/,2020-01-31,last14days
1949449,ZEDPHILIPPINES,Philippines,zed.com,https://www.sailpoint.com/blog/,2019-10-02,last14days
5332090,KOGIT,Germany,kogit.de,https://www.sailpoint.com/,2020-06-26,last14days


In [3]:
sense6['visiteddate'].sort_values()

0          2019-04-02
83590      2019-04-02
83589      2019-04-02
83586      2019-04-02
24064      2019-04-02
              ...    
5539385    2020-07-12
5539384    2020-07-12
5539383    2020-07-12
5524928    2020-07-12
5474106    2020-07-12
Name: visiteddate, Length: 5540504, dtype: object

In [202]:
### bring in adwords
adwords = pd.read_csv('Search keyword Reportv2.csv',skiprows=2)
adwords['Campaign'] = adwords['Campaign'].astype('str').str.lower()
adwords['Ad group'] = adwords['Ad group'].astype('str').str.lower()
adwords['Query'] = adwords['Search keyword'].astype('str').str.lower()
adwords['Query']=adwords['Query'].apply(lambda x: x.replace('+',''))
adwords['Start date'] = pd.to_datetime(adwords['Start date'])
adwords = adwords.reset_index(drop=True)
adwords.sample(5)

,Search keyword,Search keyword status,Search keyword match type,Campaign,Campaign status,Ad group,Keyword max CPC,Currency,Keyword/Placement,Start date,End date,Clicks,Impressions,CTR,Avg. CPC,Cost,Impr. (Abs. Top) %,Impr. (Top) %,Conversions,View-through conv.,Cost / conv.,Conv. rate,Avg. CPM,Query
12669,+passwordless +authentication,Eligible,Broad,authentication_ams_en,Campaign budget constrained,general_bmm,9.00,USD,+passwordless +authentication,2020-08-12,--,0,34,0.00%,0.0,0.00,4.17%,45.83%,0.0,0,0.00,0.00%,0.00,passwordless authentication
17389,Datenschutzprinzipien,Below first page bid,Phrase,sem_emea_de_nb_phrase_gdpr,Eligible,data-protection-principles,7.88,USD,Datenschutzprinzipien,2019-02-21,--,0,8,0.00%,0.0,0.00,50.00%,75.00%,0.0,0,0.00,0.00%,0.00,datenschutzprinzipien
10042,+okta +compliance +manager,Campaign paused,Broad,sem_usa_en_nb_bmm_okta,Campaign paused,compliance-manager,7.80,USD,+okta +compliance +manager,2020-02-11,--,0,0,0.00%,0.0,0.00,0.00%,0.00%,0.0,0,0.00,0.00%,0.00,okta compliance manager
13363,cloud identity services,Campaign paused,Broad,identity is everything - search (us broad),Campaign paused,a - identity is everything,8.00,USD,cloud identity services,2016-01-14,--,0,16,0.00%,0.0,0.00,0.00%,0.00%,0.0,0,0.00,0.00%,0.00,cloud identity services
38549,gdpr europa,Campaign paused,Phrase,sem_ww_en_nb_rtg_phrase_gdpr,Campaign paused,eu-gdpr,8.00,USD,gdpr europa,2020-02-10,--,0,0,0.00%,0.0,0.00,0.00%,0.00%,0.0,0,0.00,0.00%,0.00,gdpr europa


### Break apart adwords campaigns

In [203]:
### Break in to SEM and non-SEM
SEM = adwords[adwords['Campaign'].str.contains('sem_')].reset_index(drop=True)
SEM['Geo']=SEM['Campaign'].apply(lambda x: x.split('_')[1] if len(x.split('_')) > 1 else '')
SEM['Language'] = SEM['Campaign'].apply(lambda x: x.split('_')[2] if len(x.split('_')) > 2 else '')
SEM['Brand'] = SEM['Campaign'].apply(lambda x: x.split('_')[3] if len(x.split('_')) > 3 else '')
SEM['TargetingStrategy'] = SEM['Campaign'].apply(lambda x: x.split('_')[4] if len(x.split('_')) > 4 else '')
SEM['Topic'] = SEM['Campaign'].apply(lambda x: x.split('_')[5] if len(x.split('_')) > 5 else '')
SEM['Catchall'] = SEM['Campaign'].apply(lambda x: x.split('_')[6:] if len(x.split('_')) > 6 else '')
SEM.sample()

,Search keyword,Search keyword status,Search keyword match type,Campaign,Campaign status,Ad group,Keyword max CPC,Currency,Keyword/Placement,Start date,End date,Clicks,Impressions,CTR,Avg. CPC,Cost,Impr. (Abs. Top) %,Impr. (Top) %,Conversions,View-through conv.,Cost / conv.,Conv. rate,Avg. CPM,Query,Geo,Language,Brand,TargetingStrategy,Topic,Catchall
9095,+data +security +regulations,Campaign paused,Broad,sem_usa_en_nb_bmm_gdpr,Campaign paused,data-security-regulations,12.0,USD,+data +security +regulations,2018-05-07,--,0,18,0.00%,0.0,0.00,0.00%,0.00%,0.0,0,0.00,0.00%,0.00,data security regulations,usa,en,nb,bmm,gdpr,


In [204]:
### NonSEM
NonSEM = adwords[~adwords['Campaign'].str.contains('sem_')].reset_index(drop=True)
NonSEM['Topic']=NonSEM['Campaign'].apply(lambda x: x.split('_')[0] if len(x.split('_')) > 0 else '')
NonSEM['Geo']=NonSEM['Campaign'].apply(lambda x: x.split('_')[1] if len(x.split('_')) > 1 else '')
NonSEM['Language']=NonSEM['Campaign'].apply(lambda x: x.split('_')[2] if len(x.split('_')) > 2 else '')
NonSEM['TargetingStrategy']=NonSEM['Ad group'].apply(lambda x: x.split('_')[1] if len(x.split('_')) > 1 else '')
NonSEM.sample()

,Search keyword,Search keyword status,Search keyword match type,Campaign,Campaign status,Ad group,Keyword max CPC,Currency,Keyword/Placement,Start date,End date,Clicks,Impressions,CTR,Avg. CPC,Cost,Impr. (Abs. Top) %,Impr. (Top) %,Conversions,View-through conv.,Cost / conv.,Conv. rate,Avg. CPM,Query,Topic,Geo,Language,TargetingStrategy
14570,manage active directory from linux,Eligible,Phrase,iam_apac_en,Campaign budget constrained,competitors-microsoft_phrase,11.0,USD,manage active directory from linux,2020-08-12,--,0,0,0.00%,0.0,0.00,0.00%,0.00%,0.0,0,0.00,0.00%,0.00,manage active directory from linux,iam,apac,en,phrase


In [205]:
dfs =[SEM,NonSEM]
adwords = pd.concat(dfs)
adwords=adwords.fillna('')
adwords['Query']=adwords['Query'].apply(lambda x: x.replace('+',''))
adwords['CleanCampaign']=adwords['Geo']+'_'+adwords['Language']+'_'+adwords['Brand']+'_'+adwords['TargetingStrategy']

In [206]:
adwords.sample(10)

,Search keyword,Search keyword status,Search keyword match type,Campaign,Campaign status,Ad group,Keyword max CPC,Currency,Keyword/Placement,Start date,End date,Clicks,Impressions,CTR,Avg. CPC,Cost,Impr. (Abs. Top) %,Impr. (Top) %,Conversions,View-through conv.,Cost / conv.,Conv. rate,Avg. CPM,Query,Geo,Language,Brand,TargetingStrategy,Topic,Catchall,CleanCampaign
18452,healthcare identity management,Campaign paused,Exact,sem_usa_en_nb_exact_healthcare,Campaign paused,healthcare compliance,2.66,USD,healthcare identity management,2019-04-15,--,0,0,0.00%,0.00,0.00,0.00%,0.00%,0.0,0,0.00,0.00%,0.00,healthcare identity management,usa,en,nb,exact,healthcare,,usa_en_nb_exact
5840,+pam +software,Campaign paused,Broad,sem_usa_en_nb_bmm_iam,Campaign paused,educational,9.00,USD,+pam +software,2020-07-24,--,0,8,0.00%,0.00,0.00,0.00%,25.00%,0.0,0,0.00,0.00%,0.00,pam software,usa,en,nb,bmm,iam,,usa_en_nb_bmm
12517,+what +is +access +management +system,Eligible,Broad,iam_emea_en,Campaign budget constrained,general_bmm,9.00,USD,+what +is +access +management +system,2020-08-12,--,0,5,0.00%,0.00,0.00,80.00%,100.00%,0.0,0,0.00,0.00%,0.00,what is access management system,emea,en,,bmm,iam,,emea_en__bmm
17529,+time +warner +self +service +password +management +tool,Campaign paused,Broad,sem_int_en_nb_bmm_iam,Campaign paused,educational,9.00,USD,+time +warner +self +service +password +management +tool,2020-07-24,--,0,0,0.00%,0.00,0.00,0.00%,0.00%,0.0,0,0.00,0.00%,0.00,time warner self service password management tool,int,en,nb,bmm,iam,,int_en_nb_bmm
14344,identity access management best practices,Campaign paused,Broad,identity is everything - dc seminar,Campaign paused,a - identity is everything,13.33,USD,identity access management best practices,2016-02-10,2016-02-15,0,0,0.00%,0.00,0.00,0.00%,0.00%,0.0,0,0.00,0.00%,0.00,identity access management best practices,,,,,identity is everything - dc seminar,,___
7955,sailpointiq,Campaign paused,Phrase,sem_int_en_br_phrase,Campaign paused,identityiq,6.12,USD,sailpointiq,2018-10-15,--,1,13,7.69%,5.75,5.75,38.46%,61.54%,0.0,0,0.00,0.00%,442.31,sailpointiq,int,en,br,phrase,,,int_en_br_phrase
15281,what are okta solutions,Campaign paused,Phrase,sem_apac_en_nb_phrase_okta,Campaign paused,solutions,8.51,USD,what are okta solutions,2020-02-11,--,0,0,0.00%,0.00,0.00,0.00%,0.00%,0.0,0,0.00,0.00%,0.00,what are okta solutions,apac,en,nb,phrase,okta,,apac_en_nb_phrase
15511,employee password management,Eligible,Phrase,password-management_emea_en,Eligible,general_phrase,11.00,USD,employee password management,2020-08-12,--,0,0,0.00%,0.00,0.00,0.00%,0.00%,0.0,0,0.00,0.00%,0.00,employee password management,emea,en,,phrase,password-management,,emea_en__phrase
10080,gdpr free ebook,Campaign paused,Exact,sem_apac_en_nb_exact_gdpr,Campaign paused,gdpr-ebook,5.67,USD,gdpr free ebook,2018-10-24,--,0,0,0.00%,0.00,0.00,0.00%,0.00%,0.0,0,0.00,0.00%,0.00,gdpr free ebook,apac,en,nb,exact,gdpr,,apac_en_nb_exact
2354,+gartner +magic +quadrant +service +management,Campaign paused,Broad,sem_ww_en_nb_rtg_bmm_gartner,Campaign paused,gartner-service-management,4.68,USD,+gartner +magic +quadrant +service +management,2020-02-10,--,0,0,0.00%,0.00,0.00,0.00%,0.00%,0.0,0,0.00,0.00%,0.00,gartner magic quadrant service management,ww,en,nb,rtg,bmm,[gartner],ww_en_nb_rtg


##### Make a new uniform campaign
Topic=adwords[adwords['Topic']!=""].reset_index(drop=True)
Topic=Topic.fillna('')
NTopic = adwords[adwords['Topic']==""].reset_index(drop=True)
NTopic=NTopic.fillna('')
Topic['CleanCampaign']=Topic['Geo']+'_'+Topic['Language']+'_'+Topic['Brand']+'_'+Topic['TargetingStrategy']
NTopic['CleanCampaign'] = NTopic['Geo']+'_'+NTopic['Language']
dfs=[Topic,NTopic]
adwords=pd.concat(dfs).reset_index(drop=True)
adwords.sample(10)



## Break apart 6 sense campaigns

In [566]:
### Only take rows that are search
Sense6Search = sense6[(sense6['url'].astype('str').str.contains('search')) & (sense6['url'].astype('str').str.contains('googlesearch')) & (sense6['url'].astype('str').str.contains('='))].reset_index(drop=True)
Sense6Search.sample(5)

,name,country,domain,url,visiteddate,DateRange
37259,ALLEGISGROUP,United States,allegisgroup.com,https://www.sailpoint.com/partners/technology-alliances/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=eaiaiqobchmi5ph2l_pd4givcv6gch38bauceaayasaaegksefd_bwe,2019-05-30,last14days
351556,CHILDRENSNATIONALMEDICALCENTER,United States,childrensnational.org,https://www.sailpoint.com/company/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=cj0kcqjw4dr0brcxarisakunjwsx_9kkh0l1hy3qbew3aefgp2t2w6f0skrfxom8cubet9klrwelpnkaalq9ealw_wcb,2020-04-15,last14days
316777,ENVISIONHEALTHCARE,United States,envisionhealth.com,https://www.sailpoint.com/identity-library/identity-quiz/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=cj0kcqjw09hzbrdrarisag60gp-rbf6j5h64bptkeq-dfl3d2lnz01drs-bc5_fvdfsy0zddomyyr8caal4realw_wcb&elqcta=identity-library-featured,2020-03-20,last14days
271478,INFORMATICAELCORTEINGLES,Spain,ieci.es,https://www.sailpoint.com/es/identity-library/identity-as-a-service/?elqct=paidmedia&elqchannel=googlesearch_int_es_br_phrase_sailpoint-iiq&gclid=eaiaiqobchmiwd7i1_sq5givdettch3blgemeaayasaaegiq9vd_bwe,2019-12-10,last14days
217436,EXTENDEDPRESENCE,United States,extendedpresence.com,https://www.content.sailpoint.com/gartner-magic-quadrant-for-iga/?elqct=paidmedia&elqchannel=googlesearch_usa-en-nb-exact-gartner_data-governance-tools-gartner&gclid=cjwkcaiawztubrayeiwacr67odapl35crty_d-6ouqqy7ydqaje5i2sw8nvnjghodeeo6yvucg5nlrocwnwqavd_bwe,2019-11-08,last14days


In [257]:
Sense6Search[['junk','Placeholder']]=Sense6Search.url.str.split('search',n=1,expand=True)
Sense6Search[['Placeholder','junk']]=Sense6Search.Placeholder.str.split('&',n=1,expand=True)
Sense6Search['Placeholder'] = Sense6Search['Placeholder'].str.lstrip('_')
Sense6Search['Placeholder'] = Sense6Search['Placeholder'].str.lower()
Sense6Search=Sense6Search.drop(['junk'],axis=1)
Sense6Search.sample(5)   

,name,country,domain,url,visiteddate,DateRange,Placeholder
252963,BOEHRINGERINGELHEIM,Singapore,boehringer-ingelheim.com,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_apac-en-br-exact_sailpoint&gclid=eaiaiqobchmiide9p7ek5givrg4rch3zngogeaayasaaegi93pd_bwe,2019-11-27,last14days,apac-en-br-exact_sailpoint
3328,MACQUARIEGROUP,Australia,macquarie.com,https://www.sailpoint.com/products/identityiq/?elqct=paidmedia&elqchannel=googlesearch_apac-en-br-exact_sailpoint&gclid=eaiaiqobchmi2ztnj4uw4qivbi-pch2qewn5eaayasaaegjdq_d_bwe,2019-04-02,last14days,apac-en-br-exact_sailpoint
246692,COLTTECHSVC,United Kingdom,colt.net,https://info.sailpoint.com/sp_uniform_resources?elqofferbackup=pf_missingoffer&slsbackup=19-q1-sps-ww-resource-pathfactory&elqcampaignidbackup=2179&elqct=paidmedia&elqchannel=googlesearch_int-en-nb-bmm-gartner_gartner-identity-governance&gclid=eaiaiqobchmivm_tszj55qivgrttch3nga6zeaayasaaegkeifd_bwe,2019-11-20,last14days,int-en-nb-bmm-gartner_gartner-identity-governance
180980,GOODDATA1,United States,gooddata.com,https://www.sailpoint.com/company/careers/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint-technology&gclid=cj0kcqjw8svsbrdqarisahkvyqg08mc0wxtrje6p1gp0bwjth7mcalfpi4xr2kh4autguu48ep-mpekaak8sealw_wcb,2019-10-02,last14days,usa-en-br-exact_sailpoint-technology
86951,SAILPOINTTECH,United States,sailpoint.com,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=eaiaiqobchmir_giusju4wivtv7jbx39bqbneaayasaaeglcifd_bwe,2019-08-06,last14days,usa-en-br-exact_sailpoint


In [258]:
### Create a separate DF for organic direct and drop the rest
OrganicDirect = Sense6Search[(Sense6Search['Placeholder'] == "") & (Sense6Search['url'].str.contains('organicdirect'))].reset_index(drop=True)
Sense6Search = Sense6Search[(Sense6Search['Placeholder'] != "")& (~Sense6Search['url'].str.contains('organicdirect'))]


In [259]:
Sense6Search.sample(10)


,name,country,domain,url,visiteddate,DateRange,Placeholder
352128,GENERALDYNAMICSMISSIONSYSTEMS,United States,gdmissionsystems.com,https://www.sailpoint.com/company/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=eaiaiqobchmi2r2pkcyb6qivicvkch3dpqlkeaayasaaegkuj_d_bwe,2020-04-24,last14days,usa-en-br-exact_sailpoint
218443,UMCG,Netherlands,umcg.nl,https://www.sailpoint.com/integrations/okta/?elqct=paidmedia&elqchannel=googlesearch_sitelink_sailpoint-solutions&gclid=eaiaiqobchmiouvf_mrq5qivguh3ch238qozeaayasabegjoifd_bwe,2019-11-04,last14days,sitelink_sailpoint-solutions
257637,KPMG,Canada,kpmg.ca,https://www.sailpoint.com/identity-library/identity-management-and-governance-leader-forrester-wave/?elqct=paidmedia&elqchannel=googlesearch_int-en-nb-phrase-forrester_forrester-wave&gclid=eaiaiqobchmiliuax-ip5givxjyzch11qwneeaayaiaaegkhzfd_bwe,2019-12-10,last14days,int-en-nb-phrase-forrester_forrester-wave
381611,ITWC,Canada,itwc.ca,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_int-en-br-exact_sailpoint&gclid=cj0kcqjwzzj2brdvarisabs3l9ilr8ffvc1nkqb32ixvl7yclks7qhw1vlo__iyotyf1z_qh5ukdkywaahnzealw_wcb,2020-05-21,last14days,int-en-br-exact_sailpoint
14247,SVCMASTER,United States,servicemaster.com,https://www.sailpoint.com/products/identityiq/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-phrase_identityiq-sailpoint&gclid=eaiaiqobchmigp_yto6q4givwo2zch3fcqebeaayasaaeglif_d_bwe,2019-05-10,last14days,usa-en-br-phrase_identityiq-sailpoint
189872,EQUITYLIFESTYLEPROPERTIES,United States,equitylifestyle.com,https://www.sailpoint.com/solutions/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-bmm_sailpoint-solutions&gclid=eaiaiqobchmiipczkioh5qivrp_jbx2a2g7leaaybcaaegjp0vd_bwe,2019-10-16,last14days,usa-en-br-bmm_sailpoint-solutions
59588,NaN,Philippines,misys.com,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_apac-en-br-exact_sailpoint&gclid=eaiaiqobchmiqbua__ek4wivbhhtch3bua2feaayasaaegk_d_d_bwe,2019-06-28,last14days,apac-en-br-exact_sailpoint
160974,NaN,United States,aeainvestors.com,https://www.sailpoint.com/contact-us/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=cj0kcqjw5rbsbrcfarisageyrwfilhpvxqnjacnkfb1_ub3xl8iosgou6dvm9aq_nmkymxsah39tavcaajphealw_wcb,2019-09-27,last14days,usa-en-br-exact_sailpoint
394802,CHILDCAREGROUP,United States,ccgroup.org,https://www.sailpoint.com/identity-for/business/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-bmm_sailpoint-identity-iq&gclid=cjwkcajw_ll2brakeiwav2y3sq7dbu5upusuw_0g-n8hpmd3725r89-d4lseca_f1sk9qoprjnofcxoczdcqavd_bwe,2020-05-27,last14days,usa-en-br-bmm_sailpoint-identity-iq
368595,TOPAZSYSTEMS,United States,topazsystems.com,https://www.sailpoint.com/company/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=cjwkcajwh472brageiwavhvfgu2xugvdcj2axhiyako8gmrfwhzuaepnralofug-ppxxht0era8tbbocc6aqavd_bwe,2020-05-19,last14days,usa-en-br-exact_sailpoint


In [260]:
Sense6Search['Underscores']=Sense6Search.apply(lambda x: x['Placeholder'].count('_'), axis=1)
#Sense6Search['Underscores']=Sense6Search['Underscores']*Sense6Search['Placeholder'].apply(lambda x: 100 if x.find('_') > x.find('-') else 1)
Sense6Search['Hyphens']=Sense6Search.apply(lambda x: x['Placeholder'].count('-'), axis=1)

In [261]:
Sense6Search[Sense6Search['visiteddate'] > '2020-07-01'].sample(5)

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens
413523,SAILPOINTTECH,United States,sailpoint.com,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=eaiaiqobchmivnu0vpt46qivur6tbh2vmq6feaayasaaegjplfd_bwe,2020-07-08,last14days,usa-en-br-exact_sailpoint,1,3
413366,LUNDFOODHOLDINGS,United States,lfhi.com,https://www.sailpoint.com/identity-library/identity-quiz/?elqct=paidmedia&elqchannel=googlesearch_usa_en_nb_exact_id-gov_identity-governance-gartner&gclid=cj0kcqjw6pd3brdparisan8phuepzgjg1x8vzr5ixyu3jq30dfo0jffb2zlvhbxwuypblolv_nt1qdmaaomwealw_wcb,2020-07-02,last14days,usa_en_nb_exact_id-gov_identity-governance-gartner,5,3
413377,LUNDFOODHOLDINGS,United States,lfhi.com,https://www.sailpoint.com/integrations/amazon-web-services/?elqct=paidmedia&elqchannel=googlesearch_usa_en_nb_exact_id-gov_identity-governance-gartner&gclid=cj0kcqjw6pd3brdparisan8phuepzgjg1x8vzr5ixyu3jq30dfo0jffb2zlvhbxwuypblolv_nt1qdmaaomwealw_wcb,2020-07-02,last14days,usa_en_nb_exact_id-gov_identity-governance-gartner,5,3
413052,MYTEC,United Kingdom,mytecgroup.co.uk,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch_int_en_nb_bmm_id-gov_identity-governance-sailpoint&gclid=cjwkcajwrvv3brajeiwahwodm9i1vluwcxjpciuldfdf8zwsqj9muncd0nteddnwpe35-6l4jjvjmhocdf4qavd_bwe,2020-07-03,last14days,int_en_nb_bmm_id-gov_identity-governance-sailpoint,5,3
413149,LUNDFOODHOLDINGS,United States,lfhi.com,https://www.sailpoint.com/identity-library/identity-quiz/?elqct=paidmedia&elqchannel=googlesearch_usa_en_nb_exact_id-gov_identity-governance-gartner&gclid=cj0kcqjw6pd3brdparisan8phuepzgjg1x8vzr5ixyu3jq30dfo0jffb2zlvhbxwuypblolv_nt1qdmaaomwealw_wcb,2020-07-02,last14days,usa_en_nb_exact_id-gov_identity-governance-gartner,5,3


In [262]:
Sense6Search['month'] = pd.DatetimeIndex(Sense6Search['visiteddate']).month.astype('str') + '_' + pd.DatetimeIndex(Sense6Search['visiteddate']).year.astype('str')


In [263]:
### reset the index just to be safe
Sense6Search=Sense6Search.reset_index(drop=True)
Sense6Search.sample(10)

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month
153809,PWC,United Kingdom,pwc.com,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_int-en-br-exact_sailpoint&gclid=eaiaiqobchmi5vydhpua5qivxrttch3mrak5eaayasaaegkqvfd_bwe,2019-10-03,last14days,int-en-br-exact_sailpoint,1,3,10_2019
283612,PHREESIA,Canada,phreesia.com,https://www.sailpoint.com/solutions/see-more/?elqct=paidmedia&elqchannel=googlesearch_int-en-br-exact_sailpoint&gclid=cjwkcaia1rpybrareiwa1uiy8hpnxgcjkecwdqsop3mkokun4htrwzukbh80tfa4gfax_cn2it9bzrocynyqavd_bwe&elqcta=solutions,2020-02-20,last14days,int-en-br-exact_sailpoint,1,3,2_2020
171178,HERZIGOLIVER,Germany,sage.de,https://www.sailpoint.com/de/integrations/servicenow/?elqct=paidmedia&elqchannel=googlesearch_int-de-br-exact_sailpoint&gclid=eaiaiqobchmipp3rpjin5qivvud3ch1qxq61eaayasaaegj1fpd_bwe,2019-10-08,last14days,int-de-br-exact_sailpoint,1,3,10_2019
344870,NaN,United States,automationanywhere.com,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=cjwkcajw1v_0brakeiwalfkj5sqwjzisp23wn86s0zmledebstkdcwtxky_icety1kmtq-4uyqm_6xocceqqavd_bwe,2020-04-23,last14days,usa-en-br-exact_sailpoint,1,3,4_2020
29238,DS,France,digital.security,https://www.sailpoint.com/fr/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch_int_fr_br_exact_sailpoint&gclid=cjwkcajwlptmbrboeiwahqpvhz7esgl-xyuov_m1antfrataoh2igz0iq6a4vipj4hff3c3mczkjzroclaqqavd_bwe,2019-05-16,last14days,int_fr_br_exact_sailpoint,4,0,5_2019
53272,SAILPOINTTECH,United Kingdom,sailpoint.com,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_int-en-br-exact_sailpoint&gclid=eaiaiqobchmihi69n9_14givyyntch0a1gs2eaayasaaegkxkvd_bwe,2019-06-19,last14days,int-en-br-exact_sailpoint,1,3,6_2019
96897,CONCORDRETREATCENTERS,United States,concordretreat.com,https://www.sailpoint.com/company/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=cjwkcajw1_pqbrbieiwa71rmtaqbj5lyju6rxcjok6gix87tdtdgwnen6-t8c8w3taoz0anpvbs_uxocnyqqavd_bwe,2019-08-21,last14days,usa-en-br-exact_sailpoint,1,3,8_2019
4255,LIDERANETWORK,Spain,lidera.com,https://www.sailpoint.com/es/identity-library/identity-governance/?elqct=paidmedia&elqchannel=googlesearch_int_es_br_phrase_sailpoint&gclid=eaiaiqobchmi1v7xtnra4qivagbtch3haqsmeaayasaaegjvi_d_bwe,2019-04-08,last14days,int_es_br_phrase_sailpoint,4,0,4_2019
183547,LUFTHANSASYSTEMS,Germany,dlh.de,https://www.sailpoint.com/de/integrations/okta/?elqct=paidmedia&elqchannel=googlesearch_int-de-br-exact_sailpoint&gclid=eaiaiqobchmiv__fvqgb5qivwud3ch2f-ab5eaayasaaegkyy_d_bwe,2019-10-14,last14days,int-de-br-exact_sailpoint,1,3,10_2019
324288,KPMG,Australia,kpmg.com.au,https://www.sailpoint.com/identity-for/mergers-and-acquisitions/?elqct=paidmedia&elqchannel=googlesearch_apac-en-br-exact_sailpoint&gclid=cj0kcqjwsyb0brcoarisahblphgsydn5ifnnsbpzbrduqk6svnqgiekzhsgujv14-svuhzotgkpl0pwaaro5ealw_wcb,2020-03-31,last14days,apac-en-br-exact_sailpoint,1,3,3_2020


In [264]:
#### Attempt to just break it out one way and then see what happens
Sense6Search['Brand'] = ""
Sense6Search['TargetingStrategy'] = ""
Sense6Search['Topic'] = ""
Sense6Search['Catchall'] = ""
Sense6Search['Keyword'] = ""
### Break on hyphens
Sense6Search[['Geo','Language','Brand','TargetingStrategy','Catchall']]=Sense6Search.Placeholder.str.split('-',n=4,expand=True)
### split targeting stategy into strategy and topic
Sense6Search[['TargetingStrategy','Keyword']] = Sense6Search.TargetingStrategy.str.split('_',n=1,expand=True)
### Get Correct Keyword
Sense6Search['junk2']=Sense6Search['Placeholder'].str.split('_')
Sense6Search['Keyword']=Sense6Search['Keyword'].mask(Sense6Search['Underscores'] < Sense6Search['Hyphens'],Sense6Search.apply(lambda x: x['junk2'][x['Underscores']],axis=1))
Sense6Search=Sense6Search.drop('junk2',axis=1)
Sense6Search['Keyword'] = Sense6Search['Keyword'].apply(lambda x: '' if x == None else x)

Sense6Search.sample(10)

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand,TargetingStrategy,Topic,Catchall,Keyword,Geo,Language
383899,NaN,United States,hmutx.com,https://www.sailpoint.com/company/team/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=cjwkcajwt-l2bra_eiwaacx32f_vrq0mhaxhkcozm9abu3sbrhsnvg0lg8llmngn84wddqf28a6vmbocuuuqavd_bwe,2020-06-04,last14days,usa-en-br-exact_sailpoint,1,3,6_2020,br,exact,,None,sailpoint,usa,en
176871,BAYSHOREHEALTHCARE,Canada,bayshore.ca,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_int-en-br-exact_sailpoint&gclid=cj0kcqjw0idtbrc6arisaia5gwtygqhhejbyqyo-wnyafwo-9wbxwx1gvqmk_41c5hgsxkbx5k30nb4aalskealw_wcb,2019-10-11,last14days,int-en-br-exact_sailpoint,1,3,10_2019,br,exact,,None,sailpoint,int,en
372375,ROUNDROCKISD,United States,roundrockisd.org,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=eaiaiqobchmircop6pm96qivlcmzab1uighgeaayaiaaegidd_d_bwe,2020-05-23,last14days,usa-en-br-exact_sailpoint,1,3,5_2020,br,exact,,None,sailpoint,usa,en
396341,NaN,United States,krcthrive.com,https://www.sailpoint.com/contact-us/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=cjwkcajw_qb3braveiwavwq6vh3an6ehski5yarfyxlw4zkp2rf6vuncbqhve-fvuqyxjnalqagh-rocc2qqavd_bwe,2020-06-17,last14days,usa-en-br-exact_sailpoint,1,3,6_2020,br,exact,,None,sailpoint,usa,en
224181,CARNEGIEMELLONUNIV,United States,cmu.edu,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=cj0kcqiak7tubrdqarisamrrfuzj3iouygabwxo4bxh3rce6spdh-cq92fdetsat6-w85fcxafoufwaaajefealw_wcb,2019-11-14,last14days,usa-en-br-exact_sailpoint,1,3,11_2019,br,exact,,None,sailpoint,usa,en
151306,ELSEVIER,United Kingdom,reedelsevier.com,https://www.sailpoint.com/identity-library/getting-compliant-with-identity-governance-for-files/?elqct=paidmedia&elqchannel=googlesearch_int-tm-phrase-dag_data-classification&gclid=eaiaiqobchmi-ert29_r5aiv15tvch0ybqgjeaayaiaaegkg9fd_bwe,2019-09-25,last14days,int-tm-phrase-dag_data-classification,1,4,9_2019,phrase,dag,,classification,data-classification,int,tm
350114,ORGANIZEDBYELLIS,United States,organizedbyellis.com,https://www.sailpoint.com/company/team/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=eaiaiqobchmi_mc9uoqq6qivilzach2migrgeaayasaaegl0zpd_bwe,2020-04-30,last14days,usa-en-br-exact_sailpoint,1,3,4_2020,br,exact,,None,sailpoint,usa,en
29240,DS,France,digital.security,https://www.sailpoint.com/fr/?elqct=paidmedia&elqchannel=googlesearch_int_fr_br_exact_sailpoint&gclid=cjwkcajwlptmbrboeiwahqpvhz7esgl-xyuov_m1antfrataoh2igz0iq6a4vipj4hff3c3mczkjzroclaqqavd_bwe,2019-05-16,last14days,int_fr_br_exact_sailpoint,4,0,5_2019,None,None,,None,,int_fr_br_exact_sailpoint,None
281287,TRO,United States,kaiserpermanente.org,https://www.sailpoint.com/company/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=cj0kcqias67ybrc7arisaf49cdwkocxxd7pniyhglnvjbdpfsfb3aoircwww82x84sk4n20ob34ykmwaamrmealw_wcb,2020-02-19,last14days,usa-en-br-exact_sailpoint,1,3,2_2020,br,exact,,None,sailpoint,usa,en
354135,METLIFESERVICES,United Kingdom,metlife.com,https://www.sailpoint.com/company/?elqct=paidmedia&elqchannel=googlesearch_int-en-br-exact_sailpoint&gclid=cjwkcajw7-p1bra2eiwaxopwa2irloyeot7e4xwdxdm34tbfbkxrgsdrzi6ojrii78ewwshapmmfibocgtuqavd_bwe,2020-05-11,last14days,int-en-br-exact_sailpoint,1,3,5_2020,br,exact,,None,sailpoint,int,en


In [266]:
### Topic
Sense6Search['Catchall'].fillna('')
Sense6Search['Catchall'] = Sense6Search['Catchall'].apply(lambda x: '' if x == None else x)
Sense6Search['Topic'] = Sense6Search['Catchall'].apply(lambda x: x.split('_')[0] if len(x.split('_')) > 1 else '')
Sense6Search['junk']=""
Sense6Search['junk'] = Sense6Search['Keyword'].apply(lambda x: x.split('_')[0] if len(x.split('_')) > 1 else '')
Sense6Search['Topic'] = Sense6Search['Topic'].mask(Sense6Search['Topic']=='',Sense6Search['junk'])
Sense6Search=Sense6Search.drop('junk',axis=1)
Sense6Search.sample(10)

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand,TargetingStrategy,Topic,Catchall,Keyword,Geo,Language
257537,SECURELOGIX,United States,securelogix.com,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=eaiaiqobchmivo6n0dzc5giver7ach0hfqiheaayasaaegiezvd_bwe,2019-12-19,last14days,usa-en-br-exact_sailpoint,1,3,12_2019,br,exact,,,sailpoint,usa,en
39525,TRINET,United States,trinet.com,https://www.sailpoint.com/contact-us/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=cjwkcajw8-lnbrayeiwa6eumgn3bhhrkjetdgm9frzla7-fpoovaojbsaqoebheczddzpjgtzlkycbocvwyqavd_bwe,2019-06-06,last14days,usa-en-br-exact_sailpoint,1,3,6_2019,br,exact,,,sailpoint,usa,en
91136,FUJITSU,United States,fujitsu.com,https://www.sailpoint.com/partner_form/value-added-reseller-deployment-partner/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=cj0kcqjwv8nqbrdgarisahfr9waumo8m1w-m4wirbctshrfqawd4iukbqvdm6lbczcjc68lijrtusa8aakabealw_wcb,2019-08-13,last14days,usa-en-br-exact_sailpoint,1,3,8_2019,br,exact,,,sailpoint,usa,en
314267,LATIMERGROUP,United Kingdom,latimergroup.org,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_int-en-br-exact_sailpoint&gclid=eaiaiqobchmisji76oop6aivf-dtch0mogg0eaayasaaegjypvd_bwe,2020-03-20,last14days,int-en-br-exact_sailpoint,1,3,3_2020,br,exact,,,sailpoint,int,en
83093,EXPOLOGIC,United States,expologic.com,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-phrase_sailpoint-technologies&gclid=cjwkcajw1f_pbraeeiwapp0jkmrjjxvlx95aaxbuw38oquwxoixvpmjcewar3tgaaffksqmibujmkhocjtyqavd_bwe,2019-07-30,last14days,usa-en-br-phrase_sailpoint-technologies,1,4,7_2019,br,phrase,,technologies,sailpoint-technologies,usa,en
20751,NaN,United Arab Emirates,eim.ae,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=eaiaiqobchmit4q5utaj4givelnach0oug1qeaayasaaegivlvd_bwe,2019-05-17,last14days,usa-en-br-exact_sailpoint,1,3,5_2019,br,exact,,,sailpoint,usa,en
190875,DELOITTE,Norway,deloitte.no,https://www.content.sailpoint.com/gartner-magic-quadrant-for-iga/?elqct=paidmedia&elqchannel=googlesearch_int-en-nb-exact-gartner_gartner-leaders-quadrant&gclid=eaiaiqobchmi1y7u7l2y5qivx-r3ch324gfceaayasaaegl7ppd_bwe,2019-10-23,last14days,int-en-nb-exact-gartner_gartner-leaders-quadrant,1,6,10_2019,nb,exact,gartner,gartner_gartner-leaders-quadrant,gartner-leaders-quadrant,int,en
386826,ANDOVERSENIORHIGHSCHOOL,United States,aps1.net,https://www.sailpoint.com/company/team/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-phrase_sailpoint-technologies&gclid=cj0kcqjw_ez2brcyarisajfg-ksxkn7gaclhlmbvsiltdp3sb60giw5tjt8mdtpltcrhkkkvyjub4ewaag8-ealw_wcb,2020-06-06,last14days,usa-en-br-phrase_sailpoint-technologies,1,4,6_2020,br,phrase,,technologies,sailpoint-technologies,usa,en
394727,MANTECH,United States,mantech.com,https://www.sailpoint.com/identity-library/identity-governance/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-bmm_sailpoint-governance&gclid=eaiaiqobchmiw82nwcke6givj4fabr1a2wbfeaayasaaegks1_d_bwe,2020-06-15,last14days,usa-en-br-bmm_sailpoint-governance,1,4,6_2020,br,bmm,,governance,sailpoint-governance,usa,en
280715,SAILPOINTTECH,United Kingdom,sailpoint.com,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_int-en-br-exact_sailpoint&gclid=eaiaiqobchmiy-h8zs3q5wivizntch2fvw2heaayasaaegig8fd_bwe,2020-02-24,last14days,int-en-br-exact_sailpoint,1,3,2_2020,br,exact,,,sailpoint,int,en


In [267]:
### Break out Underscores #### THIS WORKS
### Geo
Sense6Search['junk']=""
Sense6Search['Geo']=Sense6Search['Geo'].mask(Sense6Search['Underscores'] >= Sense6Search['Hyphens'],Sense6Search.Placeholder.str.split('_',n=1))
Sense6Search['junk']=Sense6Search['junk'].mask(Sense6Search['Underscores'] >= Sense6Search['Hyphens'],Sense6Search['Geo'].str[1])
Sense6Search['Geo']=Sense6Search['Geo'].mask(Sense6Search['Underscores'] >= Sense6Search['Hyphens'],Sense6Search['Geo'].str[0])

### Language
Sense6Search['Language']=Sense6Search['Language'].mask(Sense6Search['Underscores'] >= Sense6Search['Hyphens'],Sense6Search.junk.str.split('_',n=1))
Sense6Search['junk']=Sense6Search['junk'].mask(Sense6Search['Underscores'] >= Sense6Search['Hyphens'],Sense6Search['Language'].str[1])
Sense6Search['Language']=Sense6Search['Language'].mask(Sense6Search['Underscores'] >= Sense6Search['Hyphens'],Sense6Search['Language'].str[0])

### Brand
Sense6Search['Brand']=Sense6Search['Brand'].mask(Sense6Search['Underscores'] >= Sense6Search['Hyphens'],Sense6Search.junk.str.split('_',n=1))
Sense6Search['junk']=Sense6Search['junk'].mask(Sense6Search['Underscores'] >= Sense6Search['Hyphens'],Sense6Search['Brand'].str[1])
Sense6Search['Brand']=Sense6Search['Brand'].mask(Sense6Search['Underscores'] >= Sense6Search['Hyphens'],Sense6Search['Brand'].str[0])

### TargetingStrategy
Sense6Search['TargetingStrategy']=Sense6Search['TargetingStrategy'].mask(Sense6Search['Underscores'] >= Sense6Search['Hyphens'],Sense6Search.junk.str.split('_',n=1))
Sense6Search['Keyword']=Sense6Search['Keyword'].mask(Sense6Search['Underscores'] >= Sense6Search['Hyphens'],Sense6Search['TargetingStrategy'].str[1])
Sense6Search['TargetingStrategy']=Sense6Search['TargetingStrategy'].mask(Sense6Search['Underscores'] >= Sense6Search['Hyphens'],Sense6Search['TargetingStrategy'].str[0])
Sense6Search=Sense6Search.drop('junk',axis=1)


### Test to make sure underscores are good
Sense6Search[Sense6Search['Underscores'] >= Sense6Search['Hyphens']].sample(10)

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand,TargetingStrategy,Topic,Catchall,Keyword,Geo,Language
370116,SWINERTON,United States,swinerton.com,https://www.sailpoint.com/identity-library/?elqct=paidmedia&elqchannel=googlesearch_sitelink_identity-library&gclid=cjwkcajw5ij2brbdeiwa0frc9btsgv0-pmec_l6ixcbagzbheatrlswiq92n29cs4ytowkycqgkivxocci4qavd_bwe,2020-05-18,last14days,sitelink_identity-library,1,1,5_2020,NaN,NaN,,,NaN,sitelink,identity-library
175123,PARAGONFINANCE,United Kingdom,paragon-group.co.uk,https://www.sailpoint.com/solutions/access-certification/?elqct=paidmedia&elqchannel=googlesearch_sitelink_sailpoint-solutions&gclid=eaiaiqobchmi8od8gav75aivqudtch0g1wtmeaayasabegkbovd_bwe,2019-10-01,last14days,sitelink_sailpoint-solutions,1,1,10_2019,NaN,NaN,,,NaN,sitelink,sailpoint-solutions
19590,NaN,United Kingdom,nhs.net,https://www.sailpoint.com/solutions/?elqct=paidmedia&elqchannel=googlesearch_sitelink_sailpoint-solutions&gclid=eaiaiqobchmi9mk6lpay4givbobvch03uacoeaayasabeglcfvd_bwe,2019-05-13,last14days,sitelink_sailpoint-solutions,1,1,5_2019,NaN,NaN,,,NaN,sitelink,sailpoint-solutions
48150,SKILLPARTNER,France,skill-partner.com,https://www.sailpoint.com/fr/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch_int_fr_br_exact_sailpoint&gclid=eaiaiqobchmiqkqj2q_u4giva4xvch0t6q7ceaayasaaegkcvvd_bwe,2019-06-06,last14days,int_fr_br_exact_sailpoint,4,0,6_2019,br,exact,,,sailpoint,int,fr
376257,ATLANTACITY,United States,atlantaga.gov,https://www.sailpoint.com/identity-library/?elqct=paidmedia&elqchannel=googlesearch_sitelink_identity-library&gclid=cjwkcajwh472brageiwavhvfgvyawvmls0u0ovfyhzddgyicwfksdkviihxoeuxwhwbec2oqt4uvtrocgryqavd_bwe,2020-05-20,last14days,sitelink_identity-library,1,1,5_2020,NaN,NaN,,,NaN,sitelink,identity-library
393584,PROTIVITIUS,United States,protiviti.com,https://www.sailpoint.com/solutions/?elqct=paidmedia&elqchannel=googlesearch_sitelink_sailpoint-solutions&gclid=cjwkcajw5vz2brateiwabcvil1u7eadfx4ly3nm6bwuxjhzgknht0c88vsqqrhlrz4kgtxz3joo8zxocu4qqavd_bwe,2020-06-10,last14days,sitelink_sailpoint-solutions,1,1,6_2020,NaN,NaN,,,NaN,sitelink,sailpoint-solutions
17192,GAMUDALAND,Malaysia,gamuda.com.my,https://www.sailpoint.com/solutions/?elqct=paidmedia&elqchannel=googlesearch_sitelink_sailpoint-solutions&gclid=eaiaiqobchmizbtvkyef4giv1zmrch08rq0ieaayasabegi8t_d_bwe,2019-05-16,last14days,sitelink_sailpoint-solutions,1,1,5_2019,NaN,NaN,,,NaN,sitelink,sailpoint-solutions
181966,NaN,France,bbox.fr,https://www.content.sailpoint.com/fr-gartnermq?elqct=paidmedia&elqchannel=googlesearch_int_fr_br_phrase_sailpoint&gclid=eaiaiqobchmio8e66omd5qivryfvch3zrg0ceaayasaaegkuzpd_bwe,2019-10-15,last14days,int_fr_br_phrase_sailpoint,4,0,10_2019,br,phrase,,,sailpoint,int,fr
369843,SWINERTON,United States,swinerton.com,https://www.sailpoint.com/identity-library/?elqct=paidmedia&elqchannel=googlesearch_sitelink_identity-library&gclid=cjwkcajw5ij2brbdeiwa0frc9btsgv0-pmec_l6ixcbagzbheatrlswiq92n29cs4ytowkycqgkivxocci4qavd_bwe,2020-05-18,last14days,sitelink_identity-library,1,1,5_2020,NaN,NaN,,,NaN,sitelink,identity-library
166549,BOUYGUESTELECOM,France,bouyguestelecom.fr,https://www.sailpoint.com/fr/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch_int_fr_br_exact_sailpoint&gclid=cj0kcqjww7hsbrdkarisaarsit6wmhmdmghf_7rlsdfw-0yq6tyopa4eincf6al2trnxo1_6rxahwrcaanbbealw_wcb,2019-09-26,last14days,int_fr_br_exact_sailpoint,4,0,9_2019,br,exact,,,sailpoint,int,fr


In [281]:
Sense6Search[(Sense6Search['Underscores'] >= Sense6Search['Hyphens'])].sample(50)

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand,TargetingStrategy,Topic,Catchall,Keyword,Geo,Language,CleanCampaign
306613,SOUTHWESTAIRLINES,United States,wnco.com,https://www.sailpoint.com/solutions/?elqct=paidmedia&elqchannel=googlesearch_sitelink_sailpoint-solutions&gclid=eaiaiqobchmi2_roktgt6aivrdbach2y0gpieaayasabegjtu_d_bwe,2020-03-12,last14days,sitelink_sailpoint-solutions,1,1,3_2020,,sitelink,,,sailpoint-solutions,,,sailpoint-solutions_sitelink
364103,SWINERTON,United States,swinerton.com,https://www.sailpoint.com/identity-library/?elqct=paidmedia&elqchannel=googlesearch_sitelink_identity-library&gclid=cjwkcajw5ij2brbdeiwa0frc9btsgv0-pmec_l6ixcbagzbheatrlswiq92n29cs4ytowkycqgkivxocci4qavd_bwe,2020-05-18,last14days,sitelink_identity-library,1,1,5_2020,,sitelink,,,identity-library,,,identity-library_sitelink
254980,AIR,France,airfrance.fr,https://info.sailpoint.com/sp_uniform_resources?elqofferbackup=pf_missingoffer&slsbackup=19-q1-sps-ww-resource-pathfactory&elqcampaignidbackup=2179&elg=fr&submittext=s&elqct=paidmedia&elqchannel=googlesearch_int_fr_br_exact_sailpoint&gclid=eaiaiqobchmivaa_3tmt5givlp3vch21qwvveaayasaaegja6fd_bwe,2019-12-11,last14days,int_fr_br_exact_sailpoint,4,0,12_2019,br,exact,,,sailpoint,int,fr,int_fr_br_exact
368356,SWINERTON,United States,swinerton.com,https://www.sailpoint.com/identity-library/?elqct=paidmedia&elqchannel=googlesearch_sitelink_identity-library&gclid=cjwkcajw5ij2brbdeiwa0frc9btsgv0-pmec_l6ixcbagzbheatrlswiq92n29cs4ytowkycqgkivxocci4qavd_bwe,2020-05-18,last14days,sitelink_identity-library,1,1,5_2020,,sitelink,,,identity-library,,,identity-library_sitelink
147732,CHILDRENSHOSPITALANDREGIONALMEDICALCENTERHOSP,United States,seattlechildrens.org,https://www.sailpoint.com/solutions/?elqct=paidmedia&elqchannel=googlesearch_sitelink_sailpoint-solutions,2019-09-27,last14days,sitelink_sailpoint-solutions,1,1,9_2019,,sitelink,,,sailpoint-solutions,,,sailpoint-solutions_sitelink
269361,JABILCIRCUIT,Singapore,jabil.com,https://www.sailpoint.com/solutions/?elqct=paidmedia&elqchannel=googlesearch_sitelink_sailpoint-solutions&gclid=eaiaiqobchmiz5n5yuve5giv2durch3fyw9keaayasabegjqhfd_bwe,2019-12-31,last14days,sitelink_sailpoint-solutions,1,1,12_2019,,sitelink,,,sailpoint-solutions,,,sailpoint-solutions_sitelink
132655,GENERALELECTRIC,United States,ge.com,https://www.sailpoint.com/solutions/?elqct=paidmedia&elqchannel=googlesearch_sitelink_sailpoint-solutions&gclid=eaiaiqobchmir9ikgbrl5aiv7f_jbx1g7wwbeaayasabegjgm_d_bwe,2019-09-12,last14days,sitelink_sailpoint-solutions,1,1,9_2019,,sitelink,,,sailpoint-solutions,,,sailpoint-solutions_sitelink
354533,KAISERPERMANENTE,United States,kaiserpermanente.org,https://www.sailpoint.com/solutions/?elqct=paidmedia&elqchannel=googlesearch_sitelink_sailpoint-solutions&gclid=cj0kcqjw2pp1brciarisaeqv-psdyjxtfd7d_gt6wwkh6gkmwafk7z6uffeg-kehrdkecvkdu0j0fm4aaromealw_wcb,2020-05-14,last14days,sitelink_sailpoint-solutions,1,1,5_2020,,sitelink,,,sailpoint-solutions,,,sailpoint-solutions_sitelink
335802,NaN,United States,siriuscom.com,https://www.sailpoint.com/solutions/?elqct=paidmedia&elqchannel=googlesearch_sitelink_sailpoint-solutions&gclid=eaiaiqobchmii8b2xste6aiviv_jbx0zuqkueaayasabegi1ffd_bwe,2020-04-10,last14days,sitelink_sailpoint-solutions,1,1,4_2020,,sitelink,,,sailpoint-solutions,,,sailpoint-solutions_sitelink
35017,INGENIA,Spain,ingenia.es,https://www.sailpoint.com/customers/?elqct=paidmedia&elqchannel=googlesearch_sitelink_sailpoint-solutions&gclid=eaiaiqobchmi7mncus684givjrztch3xtgdoeaayasadegkvkfd_bwe,2019-05-27,last14days,sitelink_sailpoint-solutions,1,1,5_2019,,sitelink,,,sailpoint-solutions,,,sailpoint-solutions_sitelink


In [282]:
### Break out Sitelink searches
Sense6Search['Geo'] = Sense6Search['Geo'].mask(Sense6Search['Placeholder'].str.contains('sitelink'),"")
Sense6Search['Language'] = Sense6Search['Language'].mask(Sense6Search['Placeholder'].str.contains('sitelink'),"")
Sense6Search['Brand'] = Sense6Search['Brand'].mask(Sense6Search['Placeholder'].str.contains('sitelink'),"")

### split targeting stategy into strategy and topic
Sense6Search['junk2']=""
Sense6Search['junk2']=Sense6Search['Placeholder'].str.split('_')
Sense6Search['TargetingStrategy']=Sense6Search['TargetingStrategy'].mask(Sense6Search['Placeholder'].str.contains('sitelink'),Sense6Search['junk2'].str[0])
Sense6Search['Keyword']=Sense6Search['Keyword'].mask(Sense6Search['Placeholder'].str.contains('sitelink'),Sense6Search['junk2'].str[1])
Sense6Search=Sense6Search.drop('junk2',axis=1)

Sense6Search[Sense6Search['Placeholder'].str.contains('sitelink')].sample(10)

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand,TargetingStrategy,Topic,Catchall,Keyword,Geo,Language,CleanCampaign
369074,SWINERTON,United States,swinerton.com,https://www.sailpoint.com/identity-library/?elqct=paidmedia&elqchannel=googlesearch_sitelink_identity-library&gclid=cjwkcajw5ij2brbdeiwa0frc9btsgv0-pmec_l6ixcbagzbheatrlswiq92n29cs4ytowkycqgkivxocci4qavd_bwe,2020-05-18,last14days,sitelink_identity-library,1,1,5_2020,,sitelink,,,identity-library,,,identity-library_sitelink
327753,COMMONWEALTHOFMASSACHUSETTS,United States,state.ma.us,https://www.sailpoint.com/solutions/?elqct=paidmedia&elqchannel=googlesearch_sitelink_sailpoint-solutions&gclid=cjwkcajwvzv0bra8eiwad9t2vb7arykaiculats8xmb_zo0b3oj2tyyv-nhzqn_xrpopwsotkuvcjrocq_oqavd_bwe,2020-04-03,last14days,sitelink_sailpoint-solutions,1,1,4_2020,,sitelink,,,sailpoint-solutions,,,sailpoint-solutions_sitelink
176760,NaN,Sweden,teliacompany.com,https://www.sailpoint.com/partners/system-integrators-and-resellers/?elqct=paidmedia&elqchannel=googlesearch_sitelink_sailpoint-solutions&gclid=eaiaiqobchmi3-hl_cun5qivieiach0jbwrbeaayasabeglnlfd_bwe,2019-10-08,last14days,sitelink_sailpoint-solutions,1,1,10_2019,,sitelink,,,sailpoint-solutions,,,sailpoint-solutions_sitelink
369279,SWINERTON,United States,swinerton.com,https://www.sailpoint.com/identity-library/?elqct=paidmedia&elqchannel=googlesearch_sitelink_identity-library&gclid=cjwkcajw5ij2brbdeiwa0frc9btsgv0-pmec_l6ixcbagzbheatrlswiq92n29cs4ytowkycqgkivxocci4qavd_bwe,2020-05-18,last14days,sitelink_identity-library,1,1,5_2020,,sitelink,,,identity-library,,,identity-library_sitelink
267400,CUTTINGEDGETECHNICALSVC,Australia,cuttingedge.com.au,https://www.sailpoint.com/identity-library/?elqct=paidmedia&elqchannel=googlesearch_sitelink_identity-library,2019-12-24,last14days,sitelink_identity-library,1,1,12_2019,,sitelink,,,identity-library,,,identity-library_sitelink
82714,ALTEC,United States,altec.com,https://www.sailpoint.com/solutions/?elqct=paidmedia&elqchannel=googlesearch_sitelink_sailpoint-solutions&gclid=cjwkcajwg-dpbrbbeiwaev1_-anaor-t_kv7ra351jvob_cmjpr1srhy1hfub5a_j7-l3eevlk6bwbocxnsqavd_bwe,2019-07-24,last14days,sitelink_sailpoint-solutions,1,1,7_2019,,sitelink,,,sailpoint-solutions,,,sailpoint-solutions_sitelink
354105,BASSENERGYSVC,United States,bassenergyservices.com,https://www.sailpoint.com/solutions/?elqct=paidmedia&elqchannel=googlesearch_sitelink_sailpoint-solutions&gclid=cjwkcajwte71brbceiwau_v9h_u5mnsqk31t_v7igi9qvajutgtep4nqnulxbmygvxkgf-4hjkjmkbocnzcqavd_bwe,2020-05-13,last14days,sitelink_sailpoint-solutions,1,1,5_2020,,sitelink,,,sailpoint-solutions,,,sailpoint-solutions_sitelink
256709,WSO2,Sri Lanka,wso2.com,https://info.sailpoint.com/sp_uniform_resources?elqoffer=ar_gartnermq_2019&slsbackup=19-q4-sps-ww-website-gartnermq-2019&elqcampaignidbackup=2621&elqct=paidmedia&elqchannel=googlesearch_sitelink_identity-library&elqcta=menu,2019-12-18,last14days,sitelink_identity-library,1,1,12_2019,,sitelink,,,identity-library,,,identity-library_sitelink
361271,ATLANTACITY,United States,atlantaga.gov,https://www.sailpoint.com/identity-library/?elqct=paidmedia&elqchannel=googlesearch_sitelink_identity-library&gclid=cjwkcajwh472brageiwavhvfgvyawvmls0u0ovfyhzddgyicwfksdkviihxoeuxwhwbec2oqt4uvtrocgryqavd_bwe,2020-05-20,last14days,sitelink_identity-library,1,1,5_2020,,sitelink,,,identity-library,,,identity-library_sitelink
326628,DANIELHANDHIGHSCHOOL,United States,madison.k12.ct.us,https://www.sailpoint.com/company/team/?elqct=paidmedia&elqchannel=googlesearch_sitelink_sailpoint-solutions&gclid=cj0kcqjwmpb0brcbarisag7y4zao6j1wycw2wydqpmgw3djlmxwx3bqae4phkb_1brf2r0bhapsnq4eaakntealw_wcb,2020-04-02,last14days,sitelink_sailpoint-solutions,1,1,4_2020,,sitelink,,,sailpoint-solutions,,,sailpoint-solutions_sitelink


In [324]:
### Figure out the equals
Sense6Search = Sense6Search.fillna('')
Sense6Search['Topic'] = Sense6Search.apply(lambda x: x['Keyword'].split('_')[0] if len(x['Keyword'].split('_'))>1 else x['Topic'],axis=1)
Sense6Search['Keyword'] = Sense6Search.apply(lambda x: x['Keyword'].split('_')[1] if len(x['Keyword'].split('_'))>1 else x['Keyword'],axis=1)
#Sense6Search['junk2']=Sense6Search['Keyword'].str.split('_')
#Sense6Search['Topic'] = Sense6Search['Topic'].mask((Sense6Search['Topic'] == "") & (len(Sense6Search['junk2']) > 1,Sense6Search['junk2'][0]))
           

In [325]:
Sense6Search[(Sense6Search['Underscores'] == Sense6Search['Hyphens']) & (~Sense6Search['Placeholder'].astype('str').str.contains('sitelink'))]


,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand,TargetingStrategy,Topic,Catchall,Keyword,Geo,Language,CleanCampaign
7762,MOTIONPICTUREINDUSTRYPENSION&HEALTHPLANS,United States,mpiphp.org,http://webcache.googleusercontent.com/search?q=cache:https://www.sailpoint.com/,2019-04-05,last14days,?q=cache:https://www.sailpoint.com/,0,0,4_2019,,,,,,?q=cache:https://www.sailpoint.com/,,?q=cache:https://www.sailpoint.com/___
8969,POWERSCHOOLGROUP,United States,pearsonschoolsystems.com,http://webcache.googleusercontent.com/search?q=cache:https://www.sailpoint.com/,2019-04-02,last14days,?q=cache:https://www.sailpoint.com/,0,0,4_2019,,,,,,?q=cache:https://www.sailpoint.com/,,?q=cache:https://www.sailpoint.com/___
22878,,Mexico,unam.mx,https://www.sailpoint.com/es/identity-library/identity-governance/?elqct=paidmedia&elqchannel=googlesearch_int_es_nb_bmm_id-gov_gesti%c3%b3n-de-identidad-y-acceso&gclid=cj0kcqjwoinnbrddarisanbvyat3msvptad0jqnak_a2fjgyjxbcqhkyqnwylcy6uaoeusvriq99--qaau2cealw_wcb,2019-05-21,last14days,int_es_nb_bmm_id-gov_gesti%c3%b3n-de-identidad-y-acceso,5,5,5_2019,nb,bmm,id-gov,y-acceso,gesti%c3%b3n-de-identidad-y-acceso,int,es,int_es_nb_bmm
22879,,Mexico,unam.mx,https://www.sailpoint.com/es/identity-library/identity-governance/?elqct=paidmedia&elqchannel=googlesearch_int_es_nb_bmm_id-gov_gesti%c3%b3n-de-identidad-y-acceso&gclid=cj0kcqjwoinnbrddarisanbvyat3msvptad0jqnak_a2fjgyjxbcqhkyqnwylcy6uaoeusvriq99--qaau2cealw_wcb,2019-05-21,last14days,int_es_nb_bmm_id-gov_gesti%c3%b3n-de-identidad-y-acceso,5,5,5_2019,nb,bmm,id-gov,y-acceso,gesti%c3%b3n-de-identidad-y-acceso,int,es,int_es_nb_bmm
24346,KAFKAKOMMUNIKATION,Germany,kafka-kommunikation.de,https://webcache.googleusercontent.com/search?q=cache:b8aqtmmzuxyj:https://www.sailpoint.com/de/+&cd=1&hl=de&ct=clnk&gl=de&client=firefox-b-ab,2019-05-15,last14days,?q=cache:b8aqtmmzuxyj:https://www.sailpoint.com/de/+,0,0,5_2019,,,,,,?q=cache:b8aqtmmzuxyj:https://www.sailpoint.com/de/+,,?q=cache:b8aqtmmzuxyj:https://www.sailpoint.com/de/+___
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402059,WOOLWORTHS,Australia,woolworths.com.au,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch_apac_en_nb_exact_id-gov_identity-governance-and-access-management&gclid=eaiaiqobchmi88yz-ma46giv2aorch3ykaw1eaayaiaaegiiv_d_bwe,2020-07-06,last14days,apac_en_nb_exact_id-gov_identity-governance-and-access-management,5,5,7_2020,nb,exact,id-gov,access-management,identity-governance-and-access-management,apac,en,apac_en_nb_exact
402117,MAXISCOMMUNICATIONBERHAD,Malaysia,maxis.com.my,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch_apac_en_nb_exact_id-gov_identity-governance-and-access-management&gclid=eaiaiqobchmio_lrn4266givwx8rch0zoahyeaayasaaegj_zfd_bwe,2020-07-07,last14days,apac_en_nb_exact_id-gov_identity-governance-and-access-management,5,5,7_2020,nb,exact,id-gov,access-management,identity-governance-and-access-management,apac,en,apac_en_nb_exact
402118,MAXISCOMMUNICATIONBERHAD,Malaysia,maxis.com.my,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch_apac_en_nb_exact_id-gov_identity-governance-and-access-management&gclid=eaiaiqobchmio_lrn4266givwx8rch0zoahyeaayasaaegj_zfd_bwe,2020-07-07,last14days,apac_en_nb_exact_id-gov_identity-governance-and-access-management,5,5,7_2020,nb,exact,id-gov,access-management,identity-governance-and-access-management,apac,en,apac_en_nb_exact
402151,ZUELLIGPHARMAHOLDINGSPTELTD01,People's Republic of China,zuelligpharma.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch_apac_en_nb_exact_id-gov_identity-governance-and-access-management&gclid=cjwkcajwrvv3brajeiwahwodm46mo14yrat8ozvco

# Create Clean Campaign

In [415]:
### Build clean campaign
Sense6Search['CleanCampaign'] =  Sense6Search['Geo']+ '_' + Sense6Search['Language']+'_'+Sense6Search['Brand']+'_'+Sense6Search['TargetingStrategy']
Sense6Search['CleanCampaign'] = Sense6Search['CleanCampaign'].mask(Sense6Search['Placeholder'].str.contains('sitelink'),Sense6Search['Keyword']+'_'+Sense6Search['TargetingStrategy'])
Sense6Search.sample(10)

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand,TargetingStrategy,Topic,Catchall,Keyword,Geo,Language,CleanCampaign
176746,ESSEXPOLICE,United Kingdom,pnn.police.uk,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_int-en-br-exact_sailpoint&gclid=eaiaiqobchmiivwd3nur5qivc8dech0lra7xeaayasaaeglbtfd_bwe,2019-10-10,last14days,int-en-br-exact_sailpoint,1,3,10_2019,br,exact,,,sailpoint,int,en,int_en_br_exact
154605,,Singapore,mtcstaffing.sg,https://www.sailpoint.com/customers/?elqct=paidmedia&elqchannel=googlesearch_apac-en-br-exact_sailpoint&gclid=cj0kcqjwokzsbrc5arisaitcwxezcxyxmsdjsqynayhk9m7szdtx7z0dxlofluyn6puhskk_z-993tmaasimealw_wcb,2019-09-26,last14days,apac-en-br-exact_sailpoint,1,3,9_2019,br,exact,,,sailpoint,apac,en,apac_en_br_exact
170697,GUARDIANLIFEINSCOMPANYOFAMERICA,United States,glic.com,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=eaiaiqobchmi3o_u7per5qiviibach3aaga-eaayasaaeglpg_d_bwe,2019-10-10,last14days,usa-en-br-exact_sailpoint,1,3,10_2019,br,exact,,,sailpoint,usa,en,usa_en_br_exact
260787,JOHBERENBERGGOSSLER,United Kingdom,berenberg.de,https://www.sailpoint.com/de/customers/humana/?elqct=paidmedia&elqchannel=googlesearch_int-de-br-exact_sailpoint&gclid=eaiaiqobchmik9ssvkfb5givdj53ch3b4qpreaayasaaegkpvpd_bwe,2019-12-19,last14days,int-de-br-exact_sailpoint,1,3,12_2019,br,exact,,,sailpoint,int,de,int_de_br_exact
400395,,United States,twinlode.com,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=eaiaiqobchmirju8-dol6givtdbach3okwpweaayasaaegif0vd_bwe,2020-06-18,last14days,usa-en-br-exact_sailpoint,1,3,6_2020,br,exact,,,sailpoint,usa,en,usa_en_br_exact
71446,,Spain,segurcaixaadeslas.es,https://www.sailpoint.com/es/identity-library/identity-governance/?elqct=paidmedia&elqchannel=googlesearch_int_es_br_phrase_sailpoint&gclid=cj0kcqjwjrvpbrc0arisafrfuv_b1qc097cyfq8gv1mz77isdldfpkntic52twzmupdpxmwb2nftsc8aareqealw_wcb,2019-07-17,last14days,int_es_br_phrase_sailpoint,4,0,7_2019,br,phrase,,,sailpoint,int,es,int_es_br_phrase
326900,SPOONRIVERCOLLEGE,United States,src.edu,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=eaiaiqobchmi_e_c0f7w6aivenvach0yuqwheaayaiaaegkcpfd_bwe,2020-04-07,last14days,usa-en-br-exact_sailpoint,1,3,4_2020,br,exact,,,sailpoint,usa,en,usa_en_br_exact
278980,VISA,United States,visa.com,https://www.sailpoint.com/solutions/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=eaiaiqobchmixpjy6pjc5wiv0rx9ch0mualceaayasaaegkhlvd_bwe,2020-02-18,last14days,usa-en-br-exact_sailpoint,1,3,2_2020,br,exact,,,sailpoint,usa,en,usa_en_br_exact
319477,IBERIABANK,United States,iberiabank.com,https://www.sailpoint.com/solutions/identityiq/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-bmm_sailpoint-iiq&gclid=eaiaiqobchmimyjs7ai76aivl6dsch2wdqo1eaayasaaegk7d_d_bwe,2020-03-27,last14days,usa-en-br-bmm_sailpoint-iiq,1,4,3_2020,br,bmm,,iiq,sailpoint-iiq,usa,en,usa_en_br_bmm
255062,PASSIONRIVERFILMS,United States,passionriver.com,https://www.sailpoint.com/customers/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=cjwkcaialajvbrb_eiwa4vaqih-efjwsh8wz2pvsdfnpcenbzbvr8eco3zqbc06ywt5nat0u5jlashoc1wqqavd_bwe,2019-12-06,last14days,usa-en-br-exact_sailpoint,1,3,12_2019,br,exact,,,sailpoint,usa,en,usa_en_br_exact


# Any random extra problem children before the match?

In [237]:

### change "Topic" to "Keyword"
### If googlesearch_int-en-nb-phrase-gartner_quadrant-magic-gartner for example, 'Quadrant-magic-gartner' is the keyword. 
### so just take everything after the final '_'

### Go ahead and remove the '-' or '_' from keywords to make the matching easier with adwords



# Try to make a match?


In [416]:
adwords['Query']=adwords['Query'].astype('str').str.lower()
Sense6Search['Keyword']=Sense6Search['Keyword'].str.lower().replace(' ','-')
Sense6Search['Keyword'] = Sense6Search['Keyword'].apply(lambda x: x[:-1] if x[-1:] == '-' else x)
adwords['Query']=adwords['Query'].astype('str').str.replace(' ','-')

In [272]:
adwords.sample(10)

,Search keyword,Search keyword status,Search keyword match type,Campaign,Campaign status,Ad group,Keyword max CPC,Currency,Keyword/Placement,Start date,End date,Clicks,Impressions,CTR,Avg. CPC,Cost,Impr. (Abs. Top) %,Impr. (Top) %,Conversions,View-through conv.,Cost / conv.,Conv. rate,Avg. CPM,Query,Geo,Language,Brand,TargetingStrategy,Topic,Catchall,CleanCampaign
8406,rsa identity governance and lifecycle,Campaign paused,Phrase,sem_usa_en_nb_phrase_id-gov,Campaign paused,competitor,41.30,USD,rsa identity governance and lifecycle,2018-05-06,--,1,48,2.08%,40.2,40.20,47.92%,52.08%,0.0,0,0.00,0.00%,837.50,rsa-identity-governance-and-lifecycle,usa,en,nb,phrase,id-gov,,usa_en_nb_phrase
8515,+expensable,Below first page bid,Broad,application_emea_en,Eligible,general_bmm,9.00,USD,+expensable,2020-08-12,--,0,0,0.00%,0.0,0.00,0.00%,0.00%,0.0,0,0.00,0.00%,0.00,expensable,emea,en,,bmm,application,,emea_en__bmm
9222,windows event id list,Eligible,Phrase,id-gov_emea_en,Eligible,competitors-microsoft_phrase,11.00,USD,windows event id list,2020-08-12,--,0,0,0.00%,0.0,0.00,0.00%,0.00%,0.0,0,0.00,0.00%,0.00,windows-event-id-list,emea,en,,phrase,id-gov,,emea_en__phrase
1631,+onboarding +offboarding,Eligible,Broad,onboarding-offboarding_emea_en,Eligible,general_bmm,9.00,USD,+onboarding +offboarding,2020-08-12,--,17,304,5.59%,8.7,147.82,48.03%,91.78%,0.0,0,0.00,0.00%,486.25,onboarding-offboarding,emea,en,,bmm,onboarding-offboarding,,emea_en__bmm
18840,identity governance suite,Campaign paused,Exact,sem_apac_en_nb_exact_id-gov,Campaign paused,educational,6.00,USD,identity governance suite,2018-10-24,--,0,0,0.00%,0.0,0.00,0.00%,0.00%,0.0,0,0.00,0.00%,0.00,identity-governance-suite,apac,en,nb,exact,id-gov,,apac_en_nb_exact
4728,security rule hipaa,Campaign paused,Exact,sem_ww_en_nb_rtg_exact_healthcare,Campaign paused,hipaa security,2.66,USD,security rule hipaa,2020-02-10,--,0,0,0.00%,0.0,0.00,0.00%,0.00%,0.0,0,0.00,0.00%,0.00,security-rule-hipaa,ww,en,nb,rtg,exact,[healthcare],ww_en_nb_rtg
157,gdpr free downloads,Campaign paused,Phrase,sem_usa_en_nb_phrase_gdpr,Campaign paused,gdpr-download,8.00,USD,gdpr free downloads,2018-05-07,--,0,5,0.00%,0.0,0.00,0.00%,0.00%,0.0,0,0.00,0.00%,0.00,gdpr-free-downloads,usa,en,nb,phrase,gdpr,,usa_en_nb_phrase
15647,azure ad password manager,Below first page bid,Exact,password-management_apac_en,Eligible,competitors-microsoft_exact,13.00,USD,azure ad password manager,2020-08-12,--,0,0,0.00%,0.0,0.00,0.00%,0.00%,0.0,0,0.00,0.00%,0.00,azure-ad-password-manager,apac,en,,exact,password-management,,apac_en__exact
8915,yubikey microsoft mfa,Eligible,Phrase,authentication_emea_en,Campaign budget constrained,competitors-microsoft_phrase,11.00,USD,yubikey microsoft mfa,2020-08-12,--,0,0,0.00%,0.0,0.00,0.00%,0.00%,0.0,0,0.00,0.00%,0.00,yubikey-microsoft-mfa,emea,en,,phrase,authentication,,emea_en__phrase
19176,+importance +of +identity +governance +and +administration,Campaign paused,Broad,sem_usa_en_nb_bmm_id-gov,Campaign paused,educational,6.48,USD,+importance +of +identity +governance +and +administration,2018-05-06,--,0,0,0.00%,0.0,0.00,0.00%,0.00%,0.0,0,0.00,0.00%,0.00,importance-of-identity-governance-and-administration,usa,en,nb,bmm,id-gov,,usa_en_nb_bmm


In [326]:
SUPERJOIN = Sense6Search.merge(adwords,how="left",left_on=["CleanCampaign",'Topic','Keyword'], right_on=['CleanCampaign','Topic','Query'], indicator=True)
print("SUPERJOIN complete")
SUPERJOIN.sample(10)

SUPERJOIN complete


,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand_x,TargetingStrategy_x,Topic,Catchall_x,Keyword,Geo_x,Language_x,CleanCampaign,Search keyword,Search keyword status,Search keyword match type,Campaign,Campaign status,Ad group,Keyword max CPC,Currency,Keyword/Placement,Start date,End date,Clicks,Impressions,CTR,Avg. CPC,Cost,Impr. (Abs. Top) %,Impr. (Top) %,Conversions,View-through conv.,Cost / conv.,Conv. rate,Avg. CPM,Query,Geo_y,Language_y,Brand_y,TargetingStrategy_y,Catchall_y,_merge
352398,ACESOLIDWASTE,United States,acesolidwaste.com,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=cj0kcqjwzn71brcoarisaf8pjfjamtcv5vxr9tiudq4ukoeo6vpeb22qjz7ze-u0mgeqkr6ebmxineiaai7xealw_wcb,2020-05-10,last14days,usa-en-br-exact_sailpoint,1,3,5_2020,br,exact,,,sailpoint,usa,en,usa_en_br_exact,sailpoint,Campaign paused,Exact,sem_usa_en_br_exact,Campaign paused,brand,2.13,USD,sailpoint,2018-05-06,--,"46,591","282,311",16.50%,1.58,"73,384.76",81.50%,99.65%,360.00,0.0,203.83,0.77%,259.94,sailpoint,usa,en,br,exact,,both
224435,CLOUDBAKERS,United States,cloudbakers.com,https://www.sailpoint.com/integrations/okta/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint-lifecycle-manager&gclid=cj0kcqiano_ubrc1arisab496iwe0v0dro2f3t3ofytfhuki8a9ebxzdv83y5jjiwkk2vecimjn9b90aare-ealw_wcb,2019-11-12,last14days,usa-en-br-exact_sailpoint-lifecycle-manager,1,5,11_2019,br,exact,,lifecycle-manager,sailpoint-lifecycle-manager,usa,en,usa_en_br_exact,sailpoint lifecycle manager,Campaign paused,Exact,sem_usa_en_br_exact,Campaign paused,lifecycle-management,2.46,USD,sailpoint lifecycle manager,2018-05-06,--,106,527,20.11%,1.97,208.48,75.30%,98.34%,0.00,0.0,0.00,0.00%,395.60,sailpoint-lifecycle-manager,usa,en,br,exact,,both
283052,PATURNPIKECOMMISSION,United States,paturnpike.com,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=eaiaiqobchmi_b-ytvtv5wiviq_ich0qowp3eaayasaaegjs4vd_bwe,2020-02-26,last14days,usa-en-br-exact_sailpoint,1,3,2_2020,br,exact,,,sailpoint,usa,en,usa_en_br_exact,sailpoint,Campaign paused,Exact,sem_usa_en_br_exact,Campaign paused,brand,2.13,USD,sailpoint,2018-05-06,--,"46,591","282,311",16.50%,1.58,"73,384.76",81.50%,99.65%,360.00,0.0,203.83,0.77%,259.94,sailpoint,usa,en,br,exact,,both
301893,COMMONSECURITIZATIONSOLUTIONS,United States,commonsecuritization.com,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=eaiaiqobchmi3khgyjiv6aivh8dich3ddguoeaayayaaegknufd_bwe,2020-03-12,last14days,usa-en-br-exact_sailpoint,1,3,3_2020,br,exact,,,sailpoint,usa,en,usa_en_br_exact,sailpoint,Campaign paused,Exact,sem_usa_en_br_exact,Campaign paused,brand,2.13,USD,sailpoint,2018-05-06,--,"46,591","282,311",16.50%,1.58,"73,384.76",81.50%,99.65%,360.00,0.0,203.83,0.77%,259.94,sailpoint,usa,en,br,exact,,both
391957,TRUHEALTH,United States,truhealth.com,https://info.sailpoint.com/sp_uniform_contactusdemorequest?elqoffer=contactus&slsbackup=sps-all-website-contactusform-0414&elqcampaignidbackup=12&submittext=s&elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=cj0kcqjwz4z3brcgarisaes_ovdn7f_3pueo_sjcbgrd4xlykbu0wb8pa_bdypzkic6yajt03tarlmmaaurpealw_wcb,2020-06-12,last14days,usa-en-br-exact_sailpoint,1,3,6_2020,br,exact,,,sailpoint,usa,en,usa_en_br_exact,sailpoint,Campaign paused,Exact,sem_usa_en_br_exact,Campaign paused,brand,2.13,USD,sailpoint,2018-05-06,--,"46,591","282,311",16.50%,1.58,"73,384.76",81.50%,99.65%,360.00,0.0,203.83,0.77%,259.94,sailpoint,usa,en,br,exact,,both
179109,OPTUS,Australia,optus.com.au,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_apac-en-br-exact_sailpoint&gclid=cj0kcqjwivbsbrdsarisadyisj9pwlfss_28xgmbhv1sao1mfx0qv3hms3aa6oxnbmveo64g8dgrijoaajcyealw_wcb,2019-10-09,last14days,apac-en-br-exact_sailpoint,1,3,10_2019,br,exact,,,sailpoint,apac,en,apac_en_br_exact,sailpoint,Campaign paused,Exact,sem_apac_en_b

In [159]:
### 163914 now 402051
163914-402051

-238137

In [274]:
len(SUPERJOIN)

403488

In [569]:
len(SUPERJOIN[SUPERJOIN['url'].str.contains('googlesearch')])

403377

In [344]:
len(SUPERJOIN[(SUPERJOIN['_merge'] != 'both') &(~SUPERJOIN['CleanCampaign'].astype('str').str.contains('sitelink')) & (~SUPERJOIN['Keyword'].astype('str').str.contains('%')) ])/len(SUPERJOIN)


0.08727767398296223

In [345]:
len(SUPERJOIN[(SUPERJOIN['_merge'] != 'both') &(~SUPERJOIN['CleanCampaign'].astype('str').str.contains('sitelink')) & (~SUPERJOIN['Keyword'].astype('str').str.contains('%')) ])

35223

In [249]:
len(SUPERJOIN)

403488

In [377]:
### what is in 6sense but not adwords?
BothFirstPass = SUPERJOIN[(SUPERJOIN['_merge'] == 'both')].reset_index(drop=True)
SiteLink = SUPERJOIN[SUPERJOIN['CleanCampaign'].astype('str').str.contains('sitelink')].reset_index(drop=True)
NonEnglish = SUPERJOIN[SUPERJOIN['Keyword'].astype('str').str.contains('%')].reset_index(drop=True)
Missing = SUPERJOIN[(SUPERJOIN['_merge'] != 'both') &(~SUPERJOIN['CleanCampaign'].astype('str').str.contains('sitelink'))& (~SUPERJOIN['Keyword'].astype('str').str.contains('%')) ].reset_index(drop=True)
Missing.drop(Missing.columns[18:],axis=1,inplace=True)

In [378]:
Missing['CleanCampaign']=Missing['CleanCampaign'].apply(lambda x: x.replace('int_','emea_'))
Missing=Missing.drop('_merge',axis=1)

KeyError: "['_merge'] not found in axis"

In [381]:
MissingJoin = Missing.merge(adwords,how="left",left_on=["CleanCampaign",'Topic','Keyword'], right_on=['CleanCampaign','Topic','Query'], indicator=True)
MissingJoin[MissingJoin['_merge'] == 'both']
BothSecondPass = MissingJoin[MissingJoin['_merge'] == 'both'].reset_index(drop=True)
StillNo = MissingJoin[MissingJoin['_merge'] != 'both'].reset_index(drop=True)

In [412]:
StillNo.drop(StillNo.columns[18:],axis=1,inplace=True)
StillNo['CleanCampaign']=StillNo['CleanCampaign'].apply(lambda x: x.replace('emea_','int_'))
StillNo['Keyword']=StillNo.apply(lambda x: x['Keyword'].replace(x['Catchall_x'],''),axis=1)
StillNo['Keyword'] = StillNo['Keyword'].apply(lambda x: x[:-1] if x[-1:] == '-' else x)
StillNo

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand_x,TargetingStrategy_x,Topic,Catchall_x,Keyword,Geo_x,Language_x,CleanCampaign
0,NATIONWIDEBUILDINGSOCIETY,United Kingdom,nationwide.co.uk,https://www.sailpoint.com/identity-management-solutions/?elqct=paidmedia&elqchannel=googlesearch_int-en-br-bmm_sailpoint-en-iam&gclid=eaiaiqobchmimod7wssx4qivrb_tch2tzqiweaayasaaegkvnvd_bwe,2019-04-02,last14days,int-en-br-bmm_sailpoint-en-iam,1,5,4_2019,br,bmm,,en-iam,sailpoint,int,en,int_en_br_bmm
1,NATIONWIDEBUILDINGSOCIETY,United Kingdom,nationwide.co.uk,https://www.sailpoint.com/identity-management-solutions/?elqct=paidmedia&elqchannel=googlesearch_int-en-br-bmm_sailpoint-en-iam&gclid=eaiaiqobchmimod7wssx4qivrb_tch2tzqiweaayasaaegkvnvd_bwe,2019-04-02,last14days,int-en-br-bmm_sailpoint-en-iam,1,5,4_2019,br,bmm,,en-iam,sailpoint,int,en,int_en_br_bmm
2,FOURTHDIMENSION,Canada,leeno.ca,https://www.sailpoint.com/services/training-schedule/?elqct=paidmedia&elqchannel=googlesearch_int-en-br-phrase_sailpoint-en-identityiq&gclid=eaiaiqobchmilbbe7uwx4qivetvach2gqwofeaayaiaaegiajpd_bwe,2019-04-04,last14days,int-en-br-phrase_sailpoint-en-identityiq,1,5,4_2019,br,phrase,,en-identityiq,sailpoint,int,en,int_en_br_phrase
3,FOURTHDIMENSION,Canada,leeno.ca,https://www.sailpoint.com/products/identityiq/?elqct=paidmedia&elqchannel=googlesearch_int-en-br-phrase_sailpoint-en-identityiq&gclid=eaiaiqobchmilbbe7uwx4qivetvach2gqwofeaayaiaaegiajpd_bwe,2019-04-02,last14days,int-en-br-phrase_sailpoint-en-identityiq,1,5,4_2019,br,phrase,,en-identityiq,sailpoint,int,en,int_en_br_phrase
4,FOURTHDIMENSION,Canada,leeno.ca,https://www.sailpoint.com/identity-library/the-ultimate-guide-to-identity-governance/?elqct=paidmedia&elqchannel=googlesearch_int-en-br-phrase_sailpoint-en-identityiq&gclid=eaiaiqobchmilbbe7uwx4qivetvach2gqwofeaayaiaaegiajpd_bwe&elqcta=popup,2019-04-02,last14days,int-en-br-phrase_sailpoint-en-identityiq,1,5,4_2019,br,phrase,,en-identityiq,sailpoint,int,en,int_en_br_phrase
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24188,鼎新電腦股份有限公司,Taiwan,dsc.com.tw,https://www.content.sailpoint.com/l/iga-for-erp?elqct=paidmedia&elqchannel=googlesearch_apac-tc-taiwan-phrase&gclid=eaiaiqobchmihpgw7psu6giva9iwch2tlwlaeaayasaaeglinpd_bwe,2020-07-02,last14days,apac-tc-taiwan-phrase,0,3,7_2020,taiwan,phrase,,,apac-tc-taiwan-phrase,apac,tc,apac_tc_taiwan_phrase
24189,鼎新電腦股份有限公司,Taiwan,dsc.com.tw,https://www.content.sailpoint.com/l/iga-for-erp?elqct=paidmedia&elqchannel=googlesearch_apac-tc-taiwan-phrase&gclid=eaiaiqobchmihpgw7psu6giva9iwch2tlwlaeaayasaaeglinpd_bwe,2020-07-02,last14days,apac-tc-taiwan-phrase,0,3,7_2020,taiwan,phrase,,,apac-tc-taiwan-phrase,apac,tc,apac_tc_taiwan_phrase
24190,,United States,cyentia.com,https://www.sailpoint.com/identity-library/identity-risk-report/?elqct=paidmedia&elqchannel=googlesearch_usa-tm-exact-tm_sailpoint&gclid=eaiaiqobchmim7-cz8zz3givzbbach3jwwoceaayasaaegkxfpd_bwe&elqcta=library,2020-07-06,last14days,usa-tm-exact-tm_sailpoint,1,3,7_2020,exact,tm,,,sailpoint,usa,tm,usa_tm_exact_tm
24191,MOUNTAINAMERICACREDITUNION,United States,macu.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch_usa_en_nb_exact_id-gov_magic-quadrant-for-identity-governance-and-administration&gclid=eaiaiqobchmi362hvjgt6givzmdach0pyqzqeaayayaaegjfppd_bwe,2020-07-01,last14days,usa_en_nb_exact_id-gov_magic-quadrant-for-identity-governance-and-administration,5,7,7_2020,quadrant,for,,identity-governance-and-administration,magic-quadrant-for,usa_en_nb_exact_id,gov_magic,usa_en_nb_exact_id_gov_magic_quadrant_for


In [414]:
len(StillNoOneJoin)

24195

In [545]:
StillNoOneJoin = StillNo.merge(adwords,how="left",left_on=["CleanCampaign",'Topic','Keyword'], right_on=['CleanCampaign','Topic','Query'], indicator=True)
DroppedDash = StillNoOneJoin[StillNoOneJoin['_merge'] == "both"].reset_index(drop=True)
NoDroppedDash = StillNoOneJoin[StillNoOneJoin['_merge'] != "both"].reset_index(drop=True)

In [546]:
len(NoDroppedDash)

6916

In [570]:
(6916)/403377

0.017145251216603822

In [548]:
NoDroppedDash.drop(NoDroppedDash.columns[18:],axis=1,inplace=True)
NoDroppedDash['Topic'] = NoDroppedDash['Topic'].apply(lambda x: '' if x == 'tm' else x)
NoDroppedDash['CleanCampaign']=NoDroppedDash['CleanCampaign'].apply(lambda x: x.replace('tm','br'))
NoDroppedDash

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand_x,TargetingStrategy_x,Topic,Catchall_x,Keyword,Geo_x,Language_x,CleanCampaign
0,OUTSOURCEDQUALITYASSUREDSVC,Philippines,outsourced.ph,https://www.sailpoint.com/identity-library/what-is-gdpr/?elqct=paidmedia&elqchannel=googlesearch_apac-nb-phrase-gdpr_gdpr-uk&gclid=eaiaiqobchmi5ig5wfll3wivjgyqch0hyagzeaayasaaegksjpd_bwe,2019-04-08,last14days,apac-nb-phrase-gdpr_gdpr-uk,1,4,4_2019,phrase,gdpr,,uk,gdpr,apac,nb,apac_nb_phrase_gdpr
1,NETBRDISTRIBUICAOINFORMATICA,Brazil,netbr.com.br,https://www.sailpoint.com/fr/blog/integrated-identity-solution/?elqct=paidmedia&elqchannel=googlesearch_usa-en-tm-phrase-tm_sailpoint-technologies&gclid=cj0kcqia7bribrd7arisabhx8acapfhrxzdbh0kfdjq4magiwn0f8e8qcirpmhxinn-nlgz3m0_of5maajveealw_wcb,2019-04-02,last14days,usa-en-tm-phrase-tm_sailpoint-technologies,1,5,4_2019,tm,phrase,,tm_sailpoint-technologies,sailpoint-technologies,usa,en,usa_en_br_phrase
2,GLOBALVISION,Canada,globalvisioninc.com,https://www.sailpoint.com/products/identitynow/?elqct=paidmedia&elqchannel=googlesearch_int-nb-phrase-gartner_gartner-idaas&gclid=eaiaiqobchmi4tpl1mo73givivcnch1eigw0eaayaiaaeglfjfd_bwe,2019-04-12,last14days,int-nb-phrase-gartner_gartner-idaas,1,4,4_2019,phrase,gartner,,idaas,gartner,int,nb,int_nb_phrase_gartner
3,GLOBALVISION,Canada,globalvisioninc.com,https://www.sailpoint.com/identity-library/identity-governance/?elqct=paidmedia&elqchannel=googlesearch_int-nb-phrase-gartner_gartner-idaas&gclid=eaiaiqobchmi4tpl1mo73givivcnch1eigw0eaayaiaaeglfjfd_bwe,2019-04-12,last14days,int-nb-phrase-gartner_gartner-idaas,1,4,4_2019,phrase,gartner,,idaas,gartner,int,nb,int_nb_phrase_gartner
4,GLOBALVISION,Canada,globalvisioninc.com,https://www.sailpoint.com/products/identitynow/?elqct=paidmedia&elqchannel=googlesearch_int-nb-phrase-gartner_gartner-idaas&gclid=eaiaiqobchmi4tpl1mo73givivcnch1eigw0eaayaiaaeglfjfd_bwe,2019-04-12,last14days,int-nb-phrase-gartner_gartner-idaas,1,4,4_2019,phrase,gartner,,idaas,gartner,int,nb,int_nb_phrase_gartner
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6911,鼎新電腦股份有限公司,Taiwan,dsc.com.tw,https://www.content.sailpoint.com/iga-for-erp/erp-compliance?lx=-fsjfq&elqct=paidmedia&elqchannel=googlesearch_apac-tc-taiwan-phrase&gclid=eaiaiqobchmihpgw7psu6giva9iwch2tlwlaeaayasaaeglinpd_bwe,2020-07-02,last14days,apac-tc-taiwan-phrase,0,3,7_2020,taiwan,phrase,,,apac-tc-taiwan-phrase,apac,tc,apac_tc_taiwan_phrase
6912,鼎新電腦股份有限公司,Taiwan,dsc.com.tw,https://www.content.sailpoint.com/l/iga-for-erp?elqct=paidmedia&elqchannel=googlesearch_apac-tc-taiwan-phrase&gclid=eaiaiqobchmihpgw7psu6giva9iwch2tlwlaeaayasaaeglinpd_bwe,2020-07-02,last14days,apac-tc-taiwan-phrase,0,3,7_2020,taiwan,phrase,,,apac-tc-taiwan-phrase,apac,tc,apac_tc_taiwan_phrase
6913,鼎新電腦股份有限公司,Taiwan,dsc.com.tw,https://www.content.sailpoint.com/l/iga-for-erp?elqct=paidmedia&elqchannel=googlesearch_apac-tc-taiwan-phrase&gclid=eaiaiqobchmihpgw7psu6giva9iwch2tlwlaeaayasaaeglinpd_bwe,2020-07-02,last14days,apac-tc-taiwan-phrase,0,3,7_2020,taiwan,phrase,,,apac-tc-taiwan-phrase,apac,tc,apac_tc_taiwan_phrase
6914,,United States,cyentia.com,https://www.sailpoint.com/identity-library/identity-risk-report/?elqct=paidmedia&elqchannel=googlesearch_usa-tm-exact-tm_sailpoint&gclid=eaiaiqobchmim7-cz8zz3givzbbach3jwwoceaayasaaegkxfpd_bwe&elqcta=library,2020-07-06,last14days,usa-tm-exact-tm_sailpoint,1,3,7_2020,exact,tm,,,sailpoint,usa,tm,usa_br_exact_br


In [549]:
NoDroppedDashJoin = NoDroppedDash.merge(adwords,how="left",left_on=["CleanCampaign",'Topic','Keyword'], right_on=['CleanCampaign','Topic','Query'], indicator=True)
BothPassThree = NoDroppedDashJoin[NoDroppedDashJoin['_merge'] == 'both' ].reset_index(drop=True)
NoPassThree = NoDroppedDashJoin[NoDroppedDashJoin['_merge'] != 'both' ].reset_index(drop=True)

len(BothPassThree)


110

In [571]:
(6806)/403377

0.01687255346735188

In [551]:
### drop TC because it's traditional chinese characters
len(NoPassThree[NoPassThree['CleanCampaign'].str.contains('_tc')])
NoPassThree = NoPassThree[~NoPassThree['CleanCampaign'].str.contains('_tc')].reset_index(drop=True)

In [552]:
NoPassThree

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand_x,TargetingStrategy_x,Topic,Catchall_x,Keyword,Geo_x,Language_x,CleanCampaign,Search keyword,Search keyword status,Search keyword match type,Campaign,Campaign status,Ad group,Keyword max CPC,Currency,Keyword/Placement,Start date,End date,Clicks,Impressions,CTR,Avg. CPC,Cost,Impr. (Abs. Top) %,Impr. (Top) %,Conversions,View-through conv.,Cost / conv.,Conv. rate,Avg. CPM,Query,Geo,Language,Brand,TargetingStrategy,Catchall,_merge
0,OUTSOURCEDQUALITYASSUREDSVC,Philippines,outsourced.ph,https://www.sailpoint.com/identity-library/what-is-gdpr/?elqct=paidmedia&elqchannel=googlesearch_apac-nb-phrase-gdpr_gdpr-uk&gclid=eaiaiqobchmi5ig5wfll3wivjgyqch0hyagzeaayasaaegksjpd_bwe,2019-04-08,last14days,apac-nb-phrase-gdpr_gdpr-uk,1,4,4_2019,phrase,gdpr,,uk,gdpr,apac,nb,apac_nb_phrase_gdpr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,GLOBALVISION,Canada,globalvisioninc.com,https://www.sailpoint.com/products/identitynow/?elqct=paidmedia&elqchannel=googlesearch_int-nb-phrase-gartner_gartner-idaas&gclid=eaiaiqobchmi4tpl1mo73givivcnch1eigw0eaayaiaaeglfjfd_bwe,2019-04-12,last14days,int-nb-phrase-gartner_gartner-idaas,1,4,4_2019,phrase,gartner,,idaas,gartner,int,nb,int_nb_phrase_gartner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,GLOBALVISION,Canada,globalvisioninc.com,https://www.sailpoint.com/identity-library/identity-governance/?elqct=paidmedia&elqchannel=googlesearch_int-nb-phrase-gartner_gartner-idaas&gclid=eaiaiqobchmi4tpl1mo73givivcnch1eigw0eaayaiaaeglfjfd_bwe,2019-04-12,last14days,int-nb-phrase-gartner_gartner-idaas,1,4,4_2019,phrase,gartner,,idaas,gartner,int,nb,int_nb_phrase_gartner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,GLOBALVISION,Canada,globalvisioninc.com,https://www.sailpoint.com/products/identitynow/?elqct=paidmedia&elqchannel=googlesearch_int-nb-phrase-gartner_gartner-idaas&gclid=eaiaiqobchmi4tpl1mo73givivcnch1eigw0eaayaiaaeglfjfd_bwe,2019-04-12,last14days,int-nb-phrase-gartner_gartner-idaas,1,4,4_2019,phrase,gartner,,idaas,gartner,int,nb,int_nb_phrase_gartner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,GLOBALVISION,Canada,globalvisioninc.com,https://www.sailpoint.com/identity-management-solutions/?elqct=paidmedia&elqchannel=googlesearch_int-nb-phrase-gartner_gartner-idaas&gclid=eaiaiqobchmi4tpl1mo73givivcnch1eigw0eaayaiaaeglfjfd_bwe,2019-04-12,last14days,int-nb-phrase-gartner_gartner-idaas,1,4,4_2019,phrase,gartner,,idaas,gartner,int,nb,int_nb_phrase_gartner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6730,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch_int_en_nb_exact_id-gov_gartner-magic-quadrant-identity-governance-and-administration-2019&gclid=eaiaiqobchmihr30x9k96givbnz3ch1rywhkeaayasaaegkyovd_bwe,2020-07-08,last14days,int_en_nb_exact_id-gov_gartner-magic-quadrant-identity-governance-and-administration-2019,5,8,7_2020,magic,quadrant,,identity-governance-and-administration-2019,gartner-magic-quadrant,int_en_nb_exact_id,gov_gartner,int_en_nb_exact_id_gov_gartner_magic_quadrant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
6731,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/

In [553]:
NoPassThree.drop(NoPassThree.columns[18:],axis=1,inplace=True)
NoPassThree['CleanCampaign']=NoPassThree.apply(lambda x: x['CleanCampaign'].replace(x['Keyword'],''),axis=1)
NoPassThree['TargetingStrategy_x'] = NoPassThree['Brand_x']
NoPassThree['Brand_x'] = NoPassThree['Language_x']
NoPassThree['CleanCampaign'] = NoPassThree['CleanCampaign'].apply(lambda x: x[:-1] if x[-1:] == '_' else x)
NoPassThree['Geo_x'] = NoPassThree.apply(lambda x: x['Placeholder'].split('_')[0] if x['Underscores'] > 1 else x['Geo_x'],axis=1)
NoPassThree['Language_x'] = NoPassThree.apply(lambda x: x['Placeholder'].split('_')[1] if x['Underscores'] > 1 else 'en',axis=1)
NoPassThree['Brand_x'] = NoPassThree.apply(lambda x: x['Placeholder'].split('_')[2] if x['Underscores'] > 1 else x['Brand_x'],axis=1)
NoPassThree['TargetingStrategy_x'] = NoPassThree.apply(lambda x: x['Placeholder'].split('_')[3] if x['Underscores'] > 2 else x['TargetingStrategy_x'],axis=1)
NoPassThree['CleanCampaign']= NoPassThree['Geo_x']+'_'+ NoPassThree['Language_x']+'_'+NoPassThree['Brand_x']+'_'+NoPassThree['TargetingStrategy_x']
NoPassThree

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand_x,TargetingStrategy_x,Topic,Catchall_x,Keyword,Geo_x,Language_x,CleanCampaign
0,OUTSOURCEDQUALITYASSUREDSVC,Philippines,outsourced.ph,https://www.sailpoint.com/identity-library/what-is-gdpr/?elqct=paidmedia&elqchannel=googlesearch_apac-nb-phrase-gdpr_gdpr-uk&gclid=eaiaiqobchmi5ig5wfll3wivjgyqch0hyagzeaayasaaegksjpd_bwe,2019-04-08,last14days,apac-nb-phrase-gdpr_gdpr-uk,1,4,4_2019,nb,phrase,,uk,gdpr,apac,en,apac_en_nb_phrase
1,GLOBALVISION,Canada,globalvisioninc.com,https://www.sailpoint.com/products/identitynow/?elqct=paidmedia&elqchannel=googlesearch_int-nb-phrase-gartner_gartner-idaas&gclid=eaiaiqobchmi4tpl1mo73givivcnch1eigw0eaayaiaaeglfjfd_bwe,2019-04-12,last14days,int-nb-phrase-gartner_gartner-idaas,1,4,4_2019,nb,phrase,,idaas,gartner,int,en,int_en_nb_phrase
2,GLOBALVISION,Canada,globalvisioninc.com,https://www.sailpoint.com/identity-library/identity-governance/?elqct=paidmedia&elqchannel=googlesearch_int-nb-phrase-gartner_gartner-idaas&gclid=eaiaiqobchmi4tpl1mo73givivcnch1eigw0eaayaiaaeglfjfd_bwe,2019-04-12,last14days,int-nb-phrase-gartner_gartner-idaas,1,4,4_2019,nb,phrase,,idaas,gartner,int,en,int_en_nb_phrase
3,GLOBALVISION,Canada,globalvisioninc.com,https://www.sailpoint.com/products/identitynow/?elqct=paidmedia&elqchannel=googlesearch_int-nb-phrase-gartner_gartner-idaas&gclid=eaiaiqobchmi4tpl1mo73givivcnch1eigw0eaayaiaaeglfjfd_bwe,2019-04-12,last14days,int-nb-phrase-gartner_gartner-idaas,1,4,4_2019,nb,phrase,,idaas,gartner,int,en,int_en_nb_phrase
4,GLOBALVISION,Canada,globalvisioninc.com,https://www.sailpoint.com/identity-management-solutions/?elqct=paidmedia&elqchannel=googlesearch_int-nb-phrase-gartner_gartner-idaas&gclid=eaiaiqobchmi4tpl1mo73givivcnch1eigw0eaayaiaaeglfjfd_bwe,2019-04-12,last14days,int-nb-phrase-gartner_gartner-idaas,1,4,4_2019,nb,phrase,,idaas,gartner,int,en,int_en_nb_phrase
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6730,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch_int_en_nb_exact_id-gov_gartner-magic-quadrant-identity-governance-and-administration-2019&gclid=eaiaiqobchmihr30x9k96givbnz3ch1rywhkeaayasaaegkyovd_bwe,2020-07-08,last14days,int_en_nb_exact_id-gov_gartner-magic-quadrant-identity-governance-and-administration-2019,5,8,7_2020,nb,exact,,identity-governance-and-administration-2019,gartner-magic-quadrant,int,en,int_en_nb_exact
6731,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch_int_en_nb_exact_id-gov_gartner-magic-quadrant-identity-governance-and-administration-2019&gclid=eaiaiqobchmihr30x9k96givbnz3ch1rywhkeaayasaaegkyovd_bwe,2020-07-08,last14days,int_en_nb_exact_id-gov_gartner-magic-quadrant-identity-governance-and-administration-2019,5,8,7_2020,nb,exact,,identity-governance-and-administration-2019,gartner-magic-quadrant,int,en,int_en_nb_exact
6732,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch_int_en_nb_exact_id-gov_gartner-magic-quadrant-identity-governance-and-administration-2019&gclid=eaiaiqobchmihr30x9k96givbnz3ch1rywhkeaayasaaegkyovd_bwe,2020-07-08,last14days,int_en_nb_exact_id-gov_gartner-magic-quadrant-identity-governance-and-administration-2019,5,8,7_2020,nb,exact,,identity-governance-and-administration-2019,gartner-magic-quadrant,int,en,int_en_nb_exact
6733,,United States,cyentia.com,https://www.sailpoint.com/identity-library/identity-risk-report/?elqct=paidmedia&elqchannel=googlesearch_usa-tm-exact-tm_sailpoint&gclid=eaiaiqobchmim7-cz8zz3givzbbach3jwwoceaayasaaegkxfpd_bwe&elqcta=library,2020-07-06,last14days,usa-tm-exact-tm_sailpoint,1,3,7_2020,tm,exact,,,sailpoint,usa,en,usa_en_tm_exact


In [554]:
NoPassThree['Keyword'] = NoPassThree.apply(lambda x: x['Placeholder'].split('_')[5] if x['Underscores'] > 4 else (x['Placeholder'].split('_')[1] if x['Underscores'] > 0 else x['Keyword']),axis=1)
NoPassThree['Topic'] = NoPassThree.apply(lambda x: x['Placeholder'].split('_')[4] if x['Underscores'] > 3 else x['Topic'],axis=1)


In [555]:
NoPassThree['CleanCampaign']= NoPassThree['CleanCampaign'].apply(lambda x: x.replace('tm','br'))

In [556]:
NoPassThree

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand_x,TargetingStrategy_x,Topic,Catchall_x,Keyword,Geo_x,Language_x,CleanCampaign
0,OUTSOURCEDQUALITYASSUREDSVC,Philippines,outsourced.ph,https://www.sailpoint.com/identity-library/what-is-gdpr/?elqct=paidmedia&elqchannel=googlesearch_apac-nb-phrase-gdpr_gdpr-uk&gclid=eaiaiqobchmi5ig5wfll3wivjgyqch0hyagzeaayasaaegksjpd_bwe,2019-04-08,last14days,apac-nb-phrase-gdpr_gdpr-uk,1,4,4_2019,nb,phrase,,uk,gdpr-uk,apac,en,apac_en_nb_phrase
1,GLOBALVISION,Canada,globalvisioninc.com,https://www.sailpoint.com/products/identitynow/?elqct=paidmedia&elqchannel=googlesearch_int-nb-phrase-gartner_gartner-idaas&gclid=eaiaiqobchmi4tpl1mo73givivcnch1eigw0eaayaiaaeglfjfd_bwe,2019-04-12,last14days,int-nb-phrase-gartner_gartner-idaas,1,4,4_2019,nb,phrase,,idaas,gartner-idaas,int,en,int_en_nb_phrase
2,GLOBALVISION,Canada,globalvisioninc.com,https://www.sailpoint.com/identity-library/identity-governance/?elqct=paidmedia&elqchannel=googlesearch_int-nb-phrase-gartner_gartner-idaas&gclid=eaiaiqobchmi4tpl1mo73givivcnch1eigw0eaayaiaaeglfjfd_bwe,2019-04-12,last14days,int-nb-phrase-gartner_gartner-idaas,1,4,4_2019,nb,phrase,,idaas,gartner-idaas,int,en,int_en_nb_phrase
3,GLOBALVISION,Canada,globalvisioninc.com,https://www.sailpoint.com/products/identitynow/?elqct=paidmedia&elqchannel=googlesearch_int-nb-phrase-gartner_gartner-idaas&gclid=eaiaiqobchmi4tpl1mo73givivcnch1eigw0eaayaiaaeglfjfd_bwe,2019-04-12,last14days,int-nb-phrase-gartner_gartner-idaas,1,4,4_2019,nb,phrase,,idaas,gartner-idaas,int,en,int_en_nb_phrase
4,GLOBALVISION,Canada,globalvisioninc.com,https://www.sailpoint.com/identity-management-solutions/?elqct=paidmedia&elqchannel=googlesearch_int-nb-phrase-gartner_gartner-idaas&gclid=eaiaiqobchmi4tpl1mo73givivcnch1eigw0eaayaiaaeglfjfd_bwe,2019-04-12,last14days,int-nb-phrase-gartner_gartner-idaas,1,4,4_2019,nb,phrase,,idaas,gartner-idaas,int,en,int_en_nb_phrase
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6730,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch_int_en_nb_exact_id-gov_gartner-magic-quadrant-identity-governance-and-administration-2019&gclid=eaiaiqobchmihr30x9k96givbnz3ch1rywhkeaayasaaegkyovd_bwe,2020-07-08,last14days,int_en_nb_exact_id-gov_gartner-magic-quadrant-identity-governance-and-administration-2019,5,8,7_2020,nb,exact,id-gov,identity-governance-and-administration-2019,gartner-magic-quadrant-identity-governance-and-administration-2019,int,en,int_en_nb_exact
6731,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch_int_en_nb_exact_id-gov_gartner-magic-quadrant-identity-governance-and-administration-2019&gclid=eaiaiqobchmihr30x9k96givbnz3ch1rywhkeaayasaaegkyovd_bwe,2020-07-08,last14days,int_en_nb_exact_id-gov_gartner-magic-quadrant-identity-governance-and-administration-2019,5,8,7_2020,nb,exact,id-gov,identity-governance-and-administration-2019,gartner-magic-quadrant-identity-governance-and-administration-2019,int,en,int_en_nb_exact
6732,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch_int_en_nb_exact_id-gov_gartner-magic-quadrant-identity-governance-and-administration-2019&gclid=eaiaiqobchmihr30x9k96givbnz3ch1rywhkeaayasaaegkyovd_bwe,2020-07-08,last14days,int_en_nb_exact_id-gov_gartner-magic-quadrant-identity-governance-and-administration-2019,5,8,7_2020,nb,exact,id-gov,identity-governance-and-administration-2019,gartner-magic-quadrant-identity-governance-and-administration-2019,int,en,int_en_nb_exact
6733,,United States,cyentia.com,https://www.sailpoint.com/identity-library/identity-risk-report/?elqct=paidmedia&elqchannel=googlesearch_usa-tm-exact-tm_sailpoint&gclid=eaiaiqobchmim7-cz8zz3givzbbach3

In [557]:
DroppedKeywordJoin = NoPassThree.merge(adwords,how="left",left_on=["CleanCampaign",'Topic','Keyword'], right_on=['CleanCampaign','Topic','Query'], indicator=True)
BothPassFour = DroppedKeywordJoin[DroppedKeywordJoin['_merge'] == 'both' ].reset_index(drop=True)
Remainder = DroppedKeywordJoin[DroppedKeywordJoin['_merge'] != 'both' ].reset_index(drop=True)
Remainder.drop(Remainder.columns[18:],axis=1,inplace=True)

In [558]:
len(BothPassFour)

56

In [559]:
Remainder['Topic']=Remainder.apply(lambda x: x['Keyword'].split('-')[0] if x['Hyphens'] > 0 else x['Topic'], axis=1)
Remainder

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand_x,TargetingStrategy_x,Topic,Catchall_x,Keyword,Geo_x,Language_x,CleanCampaign
0,OUTSOURCEDQUALITYASSUREDSVC,Philippines,outsourced.ph,https://www.sailpoint.com/identity-library/what-is-gdpr/?elqct=paidmedia&elqchannel=googlesearch_apac-nb-phrase-gdpr_gdpr-uk&gclid=eaiaiqobchmi5ig5wfll3wivjgyqch0hyagzeaayasaaegksjpd_bwe,2019-04-08,last14days,apac-nb-phrase-gdpr_gdpr-uk,1,4,4_2019,nb,phrase,gdpr,uk,gdpr-uk,apac,en,apac_en_nb_phrase
1,GLOBALVISION,Canada,globalvisioninc.com,https://www.sailpoint.com/products/identitynow/?elqct=paidmedia&elqchannel=googlesearch_int-nb-phrase-gartner_gartner-idaas&gclid=eaiaiqobchmi4tpl1mo73givivcnch1eigw0eaayaiaaeglfjfd_bwe,2019-04-12,last14days,int-nb-phrase-gartner_gartner-idaas,1,4,4_2019,nb,phrase,gartner,idaas,gartner-idaas,int,en,int_en_nb_phrase
2,GLOBALVISION,Canada,globalvisioninc.com,https://www.sailpoint.com/identity-library/identity-governance/?elqct=paidmedia&elqchannel=googlesearch_int-nb-phrase-gartner_gartner-idaas&gclid=eaiaiqobchmi4tpl1mo73givivcnch1eigw0eaayaiaaeglfjfd_bwe,2019-04-12,last14days,int-nb-phrase-gartner_gartner-idaas,1,4,4_2019,nb,phrase,gartner,idaas,gartner-idaas,int,en,int_en_nb_phrase
3,GLOBALVISION,Canada,globalvisioninc.com,https://www.sailpoint.com/products/identitynow/?elqct=paidmedia&elqchannel=googlesearch_int-nb-phrase-gartner_gartner-idaas&gclid=eaiaiqobchmi4tpl1mo73givivcnch1eigw0eaayaiaaeglfjfd_bwe,2019-04-12,last14days,int-nb-phrase-gartner_gartner-idaas,1,4,4_2019,nb,phrase,gartner,idaas,gartner-idaas,int,en,int_en_nb_phrase
4,GLOBALVISION,Canada,globalvisioninc.com,https://www.sailpoint.com/identity-management-solutions/?elqct=paidmedia&elqchannel=googlesearch_int-nb-phrase-gartner_gartner-idaas&gclid=eaiaiqobchmi4tpl1mo73givivcnch1eigw0eaayaiaaeglfjfd_bwe,2019-04-12,last14days,int-nb-phrase-gartner_gartner-idaas,1,4,4_2019,nb,phrase,gartner,idaas,gartner-idaas,int,en,int_en_nb_phrase
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6674,,Philippines,metroretail.com.ph,https://www.sailpoint.com/events/digital-events-hub/?elqct=paidmedia&elqchannel=googlesearch_apac_en_nb_phrase_events_impact_2020&gclid=cjwkcajw88v3brbfeiwapwlevzdqkdagzdcv__ew1uhxtrluaeukugdeeea35smdolyrcuzq1s_rixocvw4qavd_bwe,2020-06-24,last14days,apac_en_nb_phrase_events_impact_2020,6,0,6_2020,nb,phrase,events,,impact,apac,en,apac_en_nb_phrase
6675,,Philippines,metroretail.com.ph,https://www.sailpoint.com/events/digital-events-hub/?elqct=paidmedia&elqchannel=googlesearch_apac_en_nb_phrase_events_impact_2020&gclid=cjwkcajw88v3brbfeiwapwlevzdqkdagzdcv__ew1uhxtrluaeukugdeeea35smdolyrcuzq1s_rixocvw4qavd_bwe,2020-06-24,last14days,apac_en_nb_phrase_events_impact_2020,6,0,6_2020,nb,phrase,events,,impact,apac,en,apac_en_nb_phrase
6676,KPMGUS,United States,home.kpmg,https://webcache.googleusercontent.com/search?q=cache:hxjargl813wj:https://www.sailpoint.com/integrations/okta/+&cd=2&hl=en&ct=clnk&gl=us,2020-07-09,last14days,?q=cache:hxjargl813wj:https://www.sailpoint.com/integrations/okta/+,0,0,7_2020,,,,,,?q=cache:hxjargl813wj:https://www.sailpoint.com/integrations/okta/+,en,?q=cache:hxjargl813wj:https://www.sailpoint.com/integrations/okta/+_en__
6677,KPMGUS,United States,home.kpmg,https://webcache.googleusercontent.com/search?q=cache:hxjargl813wj:https://www.sailpoint.com/integrations/okta/+&cd=2&hl=en&ct=clnk&gl=us,2020-07-09,last14days,?q=cache:hxjargl813wj:https://www.sailpoint.com/integrations/okta/+,0,0,7_2020,,,,,,?q=cache:hxjargl813wj:https://www.sailpoint.com/integrations/okta/+,en,?q=cache:hxjargl813wj:https://www.sailpoint.com/integrations/okta/+_en__


In [593]:
RemainderJoin = Remainder.merge(adwords,how="left",left_on=["CleanCampaign",'Topic','Keyword'], right_on=['CleanCampaign','Topic','Query'], indicator=True)
BothPassFive = RemainderJoin[RemainderJoin['_merge'] == 'both'].reset_index(drop=True)
NoPassFive = RemainderJoin[RemainderJoin['_merge'] != 'both'].reset_index(drop=True)
NoPassFive.drop(NoPassFive.columns[18:],axis=1,inplace=True)
len(BothPassFive)

16

In [595]:
NoPassFive=NoPassFive[NoPassFive['url'].str.contains('googlesearch')].reset_index(drop=True)
NoPassFive['Language_x']=NoPassFive.apply(lambda x: 'de' if x['country'] == "Germany" else x['Language_x'],axis=1)

In [596]:
len(NoPassFive)/403377

0.01602966951511861

In [599]:
###manual force
NoPassFive['Keyword']=NoPassFive.apply(lambda x: 'impact-2020' if x['Keyword'] == 'impact' else x['Keyword'], axis=1)
NoPassFive['Topic']=NoPassFive.apply(lambda x: 'events' if x['Keyword'] == 'impact-2020' else x['Topic'], axis=1)
NoPassFive

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand_x,TargetingStrategy_x,Topic,Catchall_x,Keyword,Geo_x,Language_x,CleanCampaign
0,BILLOMAT,Germany,billomat.com,https://www.sailpoint.com/de/identity-library/what-is-gdpr/?elqct=paidmedia&elqchannel=googlesearch_int-de-nb-phrase-gdpr_data-protection-law&gclid=eaiaiqobchmi_uy4o6ar4qivtzptch04cgx3eaayaiaaegjmhpd_bwe,2019-04-02,last14days,int-de-nb-phrase-gdpr_data-protection-law,1,6,4_2019,de,nb,data,gdpr_data-protection-law,data-protection-law,int,de,int_en_de_nb
1,,Germany,doolloop.de,https://www.sailpoint.com/de/identity-library/what-is-gdpr/?elqct=paidmedia&elqchannel=googlesearch_int-de-nb-phrase-gdpr_data-protection-directive&gclid=cj0kcqjwyohlbrcnarisafjkj6acbhumfjkpwdkx1cynyiaeod6m0vpvepzwxaqmqmxgrmcz9j0stwsaaqmlealw_wcb,2019-04-03,last14days,int-de-nb-phrase-gdpr_data-protection-directive,1,6,4_2019,de,nb,data,gdpr_data-protection-directive,data-protection-directive,int,de,int_en_de_nb
2,GISA,Germany,enviam.de,https://www.sailpoint.com/de/identity-library/what-is-gdpr/?elqct=paidmedia&elqchannel=googlesearch_int-de-nb-exact-gdpr_general-data-protection-regulation&gclid=eaiaiqobchmi4pahzaiv4qivfc93ch14_gudeaayasaaegjvypd_bwe,2019-04-12,last14days,int-de-nb-exact-gdpr_general-data-protection-regulation,1,7,4_2019,de,nb,general,gdpr_general-data-protection-regulation,general-data-protection-regulation,int,de,int_en_de_nb
3,GISA,Germany,enviam.de,https://www.sailpoint.com/de/identity-library/what-is-gdpr/?elqct=paidmedia&elqchannel=googlesearch_int-de-nb-exact-gdpr_general-data-protection-regulation&gclid=eaiaiqobchmi4pahzaiv4qivfc93ch14_gudeaayasaaegjvypd_bwe,2019-04-12,last14days,int-de-nb-exact-gdpr_general-data-protection-regulation,1,7,4_2019,de,nb,general,gdpr_general-data-protection-regulation,general-data-protection-regulation,int,de,int_en_de_nb
4,GISA,Germany,enviam.de,https://www.sailpoint.com/de/identity-library/what-is-gdpr/?elqct=paidmedia&elqchannel=googlesearch_int-de-nb-exact-gdpr_general-data-protection-regulation&gclid=eaiaiqobchmi4pahzaiv4qivfc93ch14_gudeaayasaaegjvypd_bwe,2019-04-12,last14days,int-de-nb-exact-gdpr_general-data-protection-regulation,1,7,4_2019,de,nb,general,gdpr_general-data-protection-regulation,general-data-protection-regulation,int,de,int_en_de_nb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6461,WHANGAREIBOYSHIGHSCHOOL,New Zealand,wbhs.school.nz,https://www.sailpoint.com/events/digital-events-hub/?elqct=paidmedia&elqchannel=googlesearch_apac_en_nb_phrase_events_impact_2020&gclid=eaiaiqobchmikkxdrdam6givkn4rch3c1ahzeaayasaaegkd1pd_bwe,2020-06-19,last14days,apac_en_nb_phrase_events_impact_2020,6,0,6_2020,nb,phrase,events,,impact-2020,apac,en,apac_en_nb_phrase
6462,,Philippines,metroretail.com.ph,https://www.sailpoint.com/events/digital-events-hub/?elqct=paidmedia&elqchannel=googlesearch_apac_en_nb_phrase_events_impact_2020&gclid=cjwkcajw88v3brbfeiwapwlevzdqkdagzdcv__ew1uhxtrluaeukugdeeea35smdolyrcuzq1s_rixocvw4qavd_bwe,2020-06-24,last14days,apac_en_nb_phrase_events_impact_2020,6,0,6_2020,nb,phrase,events,,impact-2020,apac,en,apac_en_nb_phrase
6463,,Philippines,metroretail.com.ph,https://www.sailpoint.com/events/digital-events-hub/?elqct=paidmedia&elqchannel=googlesearch_apac_en_nb_phrase_events_impact_2020&gclid=cjwkcajw88v3brbfeiwapwlevzdqkdagzdcv__ew1uhxtrluaeukugdeeea35smdolyrcuzq1s_rixocvw4qavd_bwe,2020-06-24,last14days,apac_en_nb_phrase_events_impact_2020,6,0,6_2020,nb,phrase,events,,impact-2020,apac,en,apac_en_nb_phrase
6464,,Philippines,metroretail.com.ph,https://www.sailpoint.com/events/digital-events-hub/?elqct=paidmedia&elqchannel=googlesearch_apac_en_nb_phrase_events_impact_2020&gclid=cjwkcajw88v3brbfeiwapwlevzdqkdagzdcv__ew1uhxtrluaeukugdeeea35smdolyrcuzq1s_rixocvw4qavd_bwe,2020-06-24,last14days,apac_en_nb_phrase_events_impact_2020,6,0,6_2020,nb,phrase,events,,impact-2020,apac,en,apac_en_nb_phrase


In [600]:
PassFiveJoin = NoPassFive.merge(adwords,how="left",left_on=["CleanCampaign",'Topic','Keyword'], right_on=['CleanCampaign','Topic','Query'], indicator=True)
BothPassSix = PassFiveJoin[PassFiveJoin['_merge'] == 'both'].reset_index(drop=True)
NoPassSix = PassFiveJoin[PassFiveJoin['_merge'] != 'both'].reset_index(drop=True)
NoPassSix.drop(NoPassSix.columns[18:],axis=1,inplace=True)
len(BothPassSix)

50

In [609]:
NoPassSix

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand_x,TargetingStrategy_x,Topic,Catchall_x,Keyword,Geo_x,Language_x,CleanCampaign
0,BILLOMAT,Germany,billomat.com,https://www.sailpoint.com/de/identity-library/what-is-gdpr/?elqct=paidmedia&elqchannel=googlesearch_int-de-nb-phrase-gdpr_data-protection-law&gclid=eaiaiqobchmi_uy4o6ar4qivtzptch04cgx3eaayaiaaegjmhpd_bwe,2019-04-02,last14days,int-de-nb-phrase-gdpr_data-protection-law,1,6,4_2019,de,nb,gdpr,gdpr_data-protection-law,data-protection-law,int,de,int_en_de_nb
1,,Germany,doolloop.de,https://www.sailpoint.com/de/identity-library/what-is-gdpr/?elqct=paidmedia&elqchannel=googlesearch_int-de-nb-phrase-gdpr_data-protection-directive&gclid=cj0kcqjwyohlbrcnarisafjkj6acbhumfjkpwdkx1cynyiaeod6m0vpvepzwxaqmqmxgrmcz9j0stwsaaqmlealw_wcb,2019-04-03,last14days,int-de-nb-phrase-gdpr_data-protection-directive,1,6,4_2019,de,nb,gdpr,gdpr_data-protection-directive,data-protection-directive,int,de,int_en_de_nb
2,GISA,Germany,enviam.de,https://www.sailpoint.com/de/identity-library/what-is-gdpr/?elqct=paidmedia&elqchannel=googlesearch_int-de-nb-exact-gdpr_general-data-protection-regulation&gclid=eaiaiqobchmi4pahzaiv4qivfc93ch14_gudeaayasaaegjvypd_bwe,2019-04-12,last14days,int-de-nb-exact-gdpr_general-data-protection-regulation,1,7,4_2019,de,nb,gdpr,gdpr_general-data-protection-regulation,general-data-protection-regulation,int,de,int_en_de_nb
3,GISA,Germany,enviam.de,https://www.sailpoint.com/de/identity-library/what-is-gdpr/?elqct=paidmedia&elqchannel=googlesearch_int-de-nb-exact-gdpr_general-data-protection-regulation&gclid=eaiaiqobchmi4pahzaiv4qivfc93ch14_gudeaayasaaegjvypd_bwe,2019-04-12,last14days,int-de-nb-exact-gdpr_general-data-protection-regulation,1,7,4_2019,de,nb,gdpr,gdpr_general-data-protection-regulation,general-data-protection-regulation,int,de,int_en_de_nb
4,GISA,Germany,enviam.de,https://www.sailpoint.com/de/identity-library/what-is-gdpr/?elqct=paidmedia&elqchannel=googlesearch_int-de-nb-exact-gdpr_general-data-protection-regulation&gclid=eaiaiqobchmi4pahzaiv4qivfc93ch14_gudeaayasaaegjvypd_bwe,2019-04-12,last14days,int-de-nb-exact-gdpr_general-data-protection-regulation,1,7,4_2019,de,nb,gdpr,gdpr_general-data-protection-regulation,general-data-protection-regulation,int,de,int_en_de_nb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6411,H3C,People's Republic of China,h3c.com,https://www.sailpoint.com/events/digital-events-hub/?elqct=paidmedia&elqchannel=googlesearch_apac_en_nb_exact_events_microsoft_ignite_2020&gclid=eaiaiqobchmixcs_7eac6givk6ywch3_manfeaayasaaegiojvd_bwe,2020-06-15,last14days,apac_en_nb_exact_events_microsoft_ignite_2020,7,0,6_2020,nb,exact,events,,microsoft,apac,en,apac_en_nb_exact
6412,H3C,People's Republic of China,h3c.com,https://www.sailpoint.com/events/digital-events-hub/?elqct=paidmedia&elqchannel=googlesearch_apac_en_nb_exact_events_microsoft_ignite_2020&gclid=eaiaiqobchmixcs_7eac6givk6ywch3_manfeaayasaaegiojvd_bwe,2020-06-15,last14days,apac_en_nb_exact_events_microsoft_ignite_2020,7,0,6_2020,nb,exact,events,,microsoft,apac,en,apac_en_nb_exact
6413,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2020-06-18,last14days,-d1-sailpoint+gartner,0,2,6_2020,d1,sailpoint+gartner,,,-d1-sailpoint+gartner,,en,_en_d1_sailpoint+gartner
6414,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2020-06-18,last14days,-d1-sailpoint+gartner,0,2,6_2020,d1,sailpoint+gartner,,,-d1-sailpoint+gartner,,en,_en_d1_sailpoint+gartner


In [626]:
NoPassSix['Topic']=NoPassSix.apply(lambda x: x['Catchall_x'].split('_')[0] if len(x['Catchall_x'].split('_')[0]) > 1 else x['Topic'],axis=1)
NoPassSix['Language_x'] = 'en'
NoPassSix['Brand_x'] = NoPassSix.apply(lambda x: x['Placeholder'].split('-')[2] if x['Language_x'] == "de" else x['Brand_x'],axis=1)
NoPassSix['TargetingStrategy_x'] = NoPassSix.apply(lambda x: x['Placeholder'].split('-')[3] if len(x['Placeholder'].split('-')) > 3 else x['TargetingStrategy_x'],axis=1)
NoPassSix['CleanCampaign']=NoPassSix.apply(lambda x: x['Geo_x']+'_'+x['Language_x']+'_'+x['Brand_x']+'_'+x['TargetingStrategy_x'] if x['country'] == 'Germany' else x['CleanCampaign'],axis=1)
NoPassSix

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand_x,TargetingStrategy_x,Topic,Catchall_x,Keyword,Geo_x,Language_x,CleanCampaign
0,BILLOMAT,Germany,billomat.com,https://www.sailpoint.com/de/identity-library/what-is-gdpr/?elqct=paidmedia&elqchannel=googlesearch_int-de-nb-phrase-gdpr_data-protection-law&gclid=eaiaiqobchmi_uy4o6ar4qivtzptch04cgx3eaayaiaaegjmhpd_bwe,2019-04-02,last14days,int-de-nb-phrase-gdpr_data-protection-law,1,6,4_2019,nb,phrase,gdpr,gdpr_data-protection-law,data-protection-law,int,en,int_en_nb_phrase
1,,Germany,doolloop.de,https://www.sailpoint.com/de/identity-library/what-is-gdpr/?elqct=paidmedia&elqchannel=googlesearch_int-de-nb-phrase-gdpr_data-protection-directive&gclid=cj0kcqjwyohlbrcnarisafjkj6acbhumfjkpwdkx1cynyiaeod6m0vpvepzwxaqmqmxgrmcz9j0stwsaaqmlealw_wcb,2019-04-03,last14days,int-de-nb-phrase-gdpr_data-protection-directive,1,6,4_2019,nb,phrase,gdpr,gdpr_data-protection-directive,data-protection-directive,int,en,int_en_nb_phrase
2,GISA,Germany,enviam.de,https://www.sailpoint.com/de/identity-library/what-is-gdpr/?elqct=paidmedia&elqchannel=googlesearch_int-de-nb-exact-gdpr_general-data-protection-regulation&gclid=eaiaiqobchmi4pahzaiv4qivfc93ch14_gudeaayasaaegjvypd_bwe,2019-04-12,last14days,int-de-nb-exact-gdpr_general-data-protection-regulation,1,7,4_2019,nb,exact,gdpr,gdpr_general-data-protection-regulation,general-data-protection-regulation,int,en,int_en_nb_exact
3,GISA,Germany,enviam.de,https://www.sailpoint.com/de/identity-library/what-is-gdpr/?elqct=paidmedia&elqchannel=googlesearch_int-de-nb-exact-gdpr_general-data-protection-regulation&gclid=eaiaiqobchmi4pahzaiv4qivfc93ch14_gudeaayasaaegjvypd_bwe,2019-04-12,last14days,int-de-nb-exact-gdpr_general-data-protection-regulation,1,7,4_2019,nb,exact,gdpr,gdpr_general-data-protection-regulation,general-data-protection-regulation,int,en,int_en_nb_exact
4,GISA,Germany,enviam.de,https://www.sailpoint.com/de/identity-library/what-is-gdpr/?elqct=paidmedia&elqchannel=googlesearch_int-de-nb-exact-gdpr_general-data-protection-regulation&gclid=eaiaiqobchmi4pahzaiv4qivfc93ch14_gudeaayasaaegjvypd_bwe,2019-04-12,last14days,int-de-nb-exact-gdpr_general-data-protection-regulation,1,7,4_2019,nb,exact,gdpr,gdpr_general-data-protection-regulation,general-data-protection-regulation,int,en,int_en_nb_exact
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6411,H3C,People's Republic of China,h3c.com,https://www.sailpoint.com/events/digital-events-hub/?elqct=paidmedia&elqchannel=googlesearch_apac_en_nb_exact_events_microsoft_ignite_2020&gclid=eaiaiqobchmixcs_7eac6givk6ywch3_manfeaayasaaegiojvd_bwe,2020-06-15,last14days,apac_en_nb_exact_events_microsoft_ignite_2020,7,0,6_2020,nb,exact,events,,microsoft,apac,en,apac_en_nb_exact
6412,H3C,People's Republic of China,h3c.com,https://www.sailpoint.com/events/digital-events-hub/?elqct=paidmedia&elqchannel=googlesearch_apac_en_nb_exact_events_microsoft_ignite_2020&gclid=eaiaiqobchmixcs_7eac6givk6ywch3_manfeaayasaaegiojvd_bwe,2020-06-15,last14days,apac_en_nb_exact_events_microsoft_ignite_2020,7,0,6_2020,nb,exact,events,,microsoft,apac,en,apac_en_nb_exact
6413,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2020-06-18,last14days,-d1-sailpoint+gartner,0,2,6_2020,d1,sailpoint+gartner,,,-d1-sailpoint+gartner,,en,_en_d1_sailpoint+gartner
6414,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2020-06-18,last14days,-d1-sailpoint+gartner,0,2,6_2020,d1,sailpoint+gartner,,,-d1-sailpoint+gartner,,en,_en_d1_sailpoint+gartner


In [637]:
PassSixJoin = NoPassSix.merge(adwords,how="left",left_on=["CleanCampaign",'Topic','Keyword'], right_on=['CleanCampaign','Topic','Query'], indicator=True)
BothPassSeven = PassSixJoin[PassSixJoin['_merge'] == 'both'].reset_index(drop=True)
NoPassSeven = PassSixJoin[PassSixJoin['_merge'] != 'both'].reset_index(drop=True)
NoPassSeven.drop(NoPassSeven.columns[18:],axis=1,inplace=True)
len(BothPassSeven)

29

In [676]:
NoPassSeven['Keyword']=NoPassSeven['Keyword'].apply(lambda x: x.replace('-en',''))
NoPassSeven['Keyword']=NoPassSeven['Keyword'].apply(lambda x: x[1:] if len(x.split('-')) > 3 else x)
NoPassSeven['Topic']=NoPassSeven['Topic'].apply(lambda x: x.replace('-en','').replace('point-','').replace('technologies','').replace('sailcyber-','').replace('security',''))
NoPassSeven['Topic']=NoPassSeven['Topic'].apply(lambda x: '' if x == 'austin' else x)
NoPassSeven['Brand_x'] = NoPassSeven.apply(lambda x: x['Placeholder'].split('-')[2] if len(x['Placeholder'].split('-')) > 2 else x['Brand_x'],axis=1)
NoPassSeven['TargetingStrategy_x'] = NoPassSeven['TargetingStrategy_x'].apply(lambda x: x.split('_')[0] if len(x.split('_'))>1 else x)
NoPassSeven['CleanCampaign'] = NoPassSeven['Geo_x']+'_'+NoPassSeven['Language_x']+'_'+NoPassSeven['Brand_x']+'_'+NoPassSeven['TargetingStrategy_x']
NoPassSeven

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand_x,TargetingStrategy_x,Topic,Catchall_x,Keyword,Geo_x,Language_x,CleanCampaign
0,BANKOFNOVASCOTIA,Canada,scotiabank.com,https://www.sailpoint.com/company/?elqct=paidmedia&elqchannel=googlesearch_int-en-br-bmm_sail-point-en-technologies&gclid=eaiaiqobchmi7ru2s9l44qivdd0mch05haageaayasaaegiorfd_bwe,2019-04-30,last14days,int-en-br-bmm_sail-point-en-technologies,1,6,4_2019,br,bmm,,point-en-technologies,sail-point-technologies,int,en,int_en_br_bmm
1,BANKOFNOVASCOTIA,Canada,scotiabank.com,https://www.sailpoint.com/company/?elqct=paidmedia&elqchannel=googlesearch_int-en-br-bmm_sail-point-en-technologies&gclid=eaiaiqobchmi7ru2s9l44qivdd0mch05haageaayasaaegiorfd_bwe,2019-04-30,last14days,int-en-br-bmm_sail-point-en-technologies,1,6,4_2019,br,bmm,,point-en-technologies,sail-point-technologies,int,en,int_en_br_bmm
2,BANKOFNOVASCOTIA,Canada,scotiabank.com,https://www.sailpoint.com/company/?elqct=paidmedia&elqchannel=googlesearch_int-en-br-bmm_sail-point-en-technologies&gclid=eaiaiqobchmi7ru2s9l44qivdd0mch05haageaayasaaegiorfd_bwe,2019-04-30,last14days,int-en-br-bmm_sail-point-en-technologies,1,6,4_2019,br,bmm,,point-en-technologies,sail-point-technologies,int,en,int_en_br_bmm
3,BANKOFNOVASCOTIA,Canada,scotiabank.com,https://www.sailpoint.com/company/?elqct=paidmedia&elqchannel=googlesearch_int-en-br-bmm_sail-point-en-technologies&gclid=eaiaiqobchmi7ru2s9l44qivdd0mch05haageaayasaaegiorfd_bwe,2019-04-30,last14days,int-en-br-bmm_sail-point-en-technologies,1,6,4_2019,br,bmm,,point-en-technologies,sail-point-technologies,int,en,int_en_br_bmm
4,BANKOFNOVASCOTIA,Canada,scotiabank.com,https://www.sailpoint.com/company/?elqct=paidmedia&elqchannel=googlesearch_int-en-br-bmm_sail-point-en-technologies&gclid=eaiaiqobchmi7ru2s9l44qivdd0mch05haageaayasaaegiorfd_bwe,2019-04-30,last14days,int-en-br-bmm_sail-point-en-technologies,1,6,4_2019,br,bmm,,point-en-technologies,sail-point-technologies,int,en,int_en_br_bmm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6382,H3C,People's Republic of China,h3c.com,https://www.sailpoint.com/events/digital-events-hub/?elqct=paidmedia&elqchannel=googlesearch_apac_en_nb_exact_events_microsoft_ignite_2020&gclid=eaiaiqobchmixcs_7eac6givk6ywch3_manfeaayasaaegiojvd_bwe,2020-06-15,last14days,apac_en_nb_exact_events_microsoft_ignite_2020,7,0,6_2020,nb,exact,events,,microsoft,apac,en,apac_en_nb_exact
6383,H3C,People's Republic of China,h3c.com,https://www.sailpoint.com/events/digital-events-hub/?elqct=paidmedia&elqchannel=googlesearch_apac_en_nb_exact_events_microsoft_ignite_2020&gclid=eaiaiqobchmixcs_7eac6givk6ywch3_manfeaayasaaegiojvd_bwe,2020-06-15,last14days,apac_en_nb_exact_events_microsoft_ignite_2020,7,0,6_2020,nb,exact,events,,microsoft,apac,en,apac_en_nb_exact
6384,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2020-06-18,last14days,-d1-sailpoint+gartner,0,2,6_2020,sailpoint+gartner,sailpoint+gartner,,,-d1-sailpoint+gartner,,en,_en_sailpoint+gartner_sailpoint+gartner
6385,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2020-06-18,last14days,-d1-sailpoint+gartner,0,2,6_2020,sailpoint+gartner,sailpoint+gartner,,,-d1-sailpoint+gartner,,en,_en_sailpoint+gartner_sailpoint+gartner


In [695]:
PassSevenJoin = NoPassSeven.merge(adwords,how="left",left_on=["CleanCampaign",'Topic','Keyword'], right_on=['CleanCampaign','Topic','Query'], indicator=True)
BothPassEight = PassSevenJoin[PassSevenJoin['_merge'] == 'both'].reset_index(drop=True)
NoPassEight = PassSevenJoin[PassSevenJoin['_merge'] != 'both'].reset_index(drop=True)
NoPassEight.drop(NoPassEight.columns[18:],axis=1,inplace=True)
len(BothPassEight)

176

In [697]:
NoPassEight['Keyword']=NoPassEight.apply(lambda x: x['Placeholder'].split('_')[1] if x['Hyphens'] > 2 else x['Keyword'], axis=1)
NoPassEight['Keyword']=NoPassEight['Keyword'].apply(lambda x: x.replace('en-',''))
NoPassEight['CleanCampaign']=NoPassEight['CleanCampaign'].apply(lambda x: x.replace('_tm_','_br_'))
NoPassEight['CleanCampaign']=NoPassEight.apply(lambda x: x['CleanCampaign'].replace('_nb_','_br_') if x['Keyword'].count('sailpoint') > 0 else x['CleanCampaign'], axis=1)
NoPassEight['Topic']=''
NoPassEight

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand_x,TargetingStrategy_x,Topic,Catchall_x,Keyword,Geo_x,Language_x,CleanCampaign
0,HIGHMETRIC,United Kingdom,highmetric.com,https://www.sailpoint.com/products/identityiq/?elqct=paidmedia&elqchannel=googlesearch_int-en-nb-phrase-tm_sailpoint-en-identityiq&gclid=eaiaiqobchmi7raqpnux4aiv05tvch2jmwupeaayasaaegkhufd_bwe,2019-05-10,last14days,int-en-nb-phrase-tm_sailpoint-en-identityiq,1,6,5_2019,nb,phrase,,tm_sailpoint-en-identityiq,sailpoint-identityiq,int,en,int_en_br_phrase
1,BANKOFMONTREAL,Canada,bmo.com,https://www.sailpoint.com/solutions/?elqct=paidmedia&elqchannel=googlesearch_int-en-br-exact_what-does-sailpoint-en-do&gclid=eaiaiqobchmikc3sm8-r4givaz7ach3v_qbneaayaiaaeglwrpd_bwe,2019-05-10,last14days,int-en-br-exact_what-does-sailpoint-en-do,1,7,5_2019,br,exact,,does-sailpoint-en-do,what-does-sailpoint-do,int,en,int_en_br_exact
2,BANKOFMONTREAL,Canada,bmo.com,https://www.sailpoint.com/solutions/policy-management/?elqct=paidmedia&elqchannel=googlesearch_int-en-br-exact_what-does-sailpoint-en-do&gclid=eaiaiqobchmikc3sm8-r4givaz7ach3v_qbneaayaiaaeglwrpd_bwe,2019-05-10,last14days,int-en-br-exact_what-does-sailpoint-en-do,1,7,5_2019,br,exact,,does-sailpoint-en-do,what-does-sailpoint-do,int,en,int_en_br_exact
3,BANKOFMONTREAL,Canada,bmo.com,https://www.sailpoint.com/solutions/?elqct=paidmedia&elqchannel=googlesearch_int-en-br-exact_what-does-sailpoint-en-do&gclid=eaiaiqobchmikc3sm8-r4givaz7ach3v_qbneaayaiaaeglwrpd_bwe,2019-05-10,last14days,int-en-br-exact_what-does-sailpoint-en-do,1,7,5_2019,br,exact,,does-sailpoint-en-do,what-does-sailpoint-do,int,en,int_en_br_exact
4,AUSTRALIANENERGYMARKETOPERATOR,Australia,aemo.com.au,https://www.sailpoint.com/identity-library/the-ultimate-guide-to-identity-governance/?elqct=paidmedia&elqchannel=googlesearch_apac-nb-exact-tm_sailpoint&gclid=eaiaiqobchmij7sxgmiv3givw7wwch0axabfeaayasaaegkrepd_bwe&elqcta=popup,2019-05-08,last14days,apac-nb-exact-tm_sailpoint,1,3,5_2019,exact,tm,,,sailpoint,apac,en,apac_en_exact_tm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6207,H3C,People's Republic of China,h3c.com,https://www.sailpoint.com/events/digital-events-hub/?elqct=paidmedia&elqchannel=googlesearch_apac_en_nb_exact_events_microsoft_ignite_2020&gclid=eaiaiqobchmixcs_7eac6givk6ywch3_manfeaayasaaegiojvd_bwe,2020-06-15,last14days,apac_en_nb_exact_events_microsoft_ignite_2020,7,0,6_2020,nb,exact,,,microsoft,apac,en,apac_en_nb_exact
6208,H3C,People's Republic of China,h3c.com,https://www.sailpoint.com/events/digital-events-hub/?elqct=paidmedia&elqchannel=googlesearch_apac_en_nb_exact_events_microsoft_ignite_2020&gclid=eaiaiqobchmixcs_7eac6givk6ywch3_manfeaayasaaegiojvd_bwe,2020-06-15,last14days,apac_en_nb_exact_events_microsoft_ignite_2020,7,0,6_2020,nb,exact,,,microsoft,apac,en,apac_en_nb_exact
6209,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2020-06-18,last14days,-d1-sailpoint+gartner,0,2,6_2020,sailpoint+gartner,sailpoint+gartner,,,-d1-sailpoint+gartner,,en,_en_sailpoint+gartner_sailpoint+gartner
6210,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2020-06-18,last14days,-d1-sailpoint+gartner,0,2,6_2020,sailpoint+gartner,sailpoint+gartner,,,-d1-sailpoint+gartner,,en,_en_sailpoint+gartner_sailpoint+gartner


In [680]:
6212/403377

0.015399985621391403

In [745]:
PassEightJoin = NoPassEight.merge(adwords,how="left",left_on=["CleanCampaign",'Topic','Keyword'], right_on=['CleanCampaign','Topic','Query'], indicator=True)
BothPassNine = PassEightJoin[PassEightJoin['_merge'] == 'both'].reset_index(drop=True)
NoPassNine = PassEightJoin[PassEightJoin['_merge'] != 'both'].reset_index(drop=True)
NoPassNine.drop(NoPassNine.columns[18:],axis=1,inplace=True)
len(BothPassNine)

154

# You are here

In [746]:
NoPassNine

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand_x,TargetingStrategy_x,Topic,Catchall_x,Keyword,Geo_x,Language_x,CleanCampaign
0,AUSTRALIANENERGYMARKETOPERATOR,Australia,aemo.com.au,https://www.sailpoint.com/identity-library/the-ultimate-guide-to-identity-governance/?elqct=paidmedia&elqchannel=googlesearch_apac-nb-exact-tm_sailpoint&gclid=eaiaiqobchmij7sxgmiv3givw7wwch0axabfeaayasaaegkrepd_bwe&elqcta=popup,2019-05-08,last14days,apac-nb-exact-tm_sailpoint,1,3,5_2019,exact,tm,,,sailpoint,apac,en,apac_en_exact_tm
1,AUSTRALIANENERGYMARKETOPERATOR,Australia,aemo.com.au,https://www.sailpoint.com/identity-library/the-ultimate-guide-to-identity-governance/?elqct=paidmedia&elqchannel=googlesearch_apac-nb-exact-tm_sailpoint&gclid=eaiaiqobchmij7sxgmiv3givw7wwch0axabfeaayasaaegkrepd_bwe&elqcta=popup,2019-05-02,last14days,apac-nb-exact-tm_sailpoint,1,3,5_2019,exact,tm,,,sailpoint,apac,en,apac_en_exact_tm
2,AUSTRALIANENERGYMARKETOPERATOR,Australia,aemo.com.au,https://www.sailpoint.com/partners/become-partner/?elqct=paidmedia&elqchannel=googlesearch_apac-nb-exact-tm_sailpoint&gclid=eaiaiqobchmij7sxgmiv3givw7wwch0axabfeaayasaaegkrepd_bwe,2019-05-02,last14days,apac-nb-exact-tm_sailpoint,1,3,5_2019,exact,tm,,,sailpoint,apac,en,apac_en_exact_tm
3,AUSTRALIANENERGYMARKETOPERATOR,Australia,aemo.com.au,https://www.sailpoint.com/partners/become-partner/?elqct=paidmedia&elqchannel=googlesearch_apac-nb-exact-tm_sailpoint&gclid=eaiaiqobchmij7sxgmiv3givw7wwch0axabfeaayasaaegkrepd_bwe,2019-05-07,last14days,apac-nb-exact-tm_sailpoint,1,3,5_2019,exact,tm,,,sailpoint,apac,en,apac_en_exact_tm
4,AUSTRALIANENERGYMARKETOPERATOR,Australia,aemo.com.au,https://www.sailpoint.com/identity-library/the-ultimate-guide-to-identity-governance/?elqct=paidmedia&elqchannel=googlesearch_apac-nb-exact-tm_sailpoint&gclid=eaiaiqobchmij7sxgmiv3givw7wwch0axabfeaayasaaegkrepd_bwe&elqcta=popup,2019-05-07,last14days,apac-nb-exact-tm_sailpoint,1,3,5_2019,exact,tm,,,sailpoint,apac,en,apac_en_exact_tm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6053,H3C,People's Republic of China,h3c.com,https://www.sailpoint.com/events/digital-events-hub/?elqct=paidmedia&elqchannel=googlesearch_apac_en_nb_exact_events_microsoft_ignite_2020&gclid=eaiaiqobchmixcs_7eac6givk6ywch3_manfeaayasaaegiojvd_bwe,2020-06-15,last14days,apac_en_nb_exact_events_microsoft_ignite_2020,7,0,6_2020,nb,exact,,,microsoft,apac,en,apac_en_nb_exact
6054,H3C,People's Republic of China,h3c.com,https://www.sailpoint.com/events/digital-events-hub/?elqct=paidmedia&elqchannel=googlesearch_apac_en_nb_exact_events_microsoft_ignite_2020&gclid=eaiaiqobchmixcs_7eac6givk6ywch3_manfeaayasaaegiojvd_bwe,2020-06-15,last14days,apac_en_nb_exact_events_microsoft_ignite_2020,7,0,6_2020,nb,exact,,,microsoft,apac,en,apac_en_nb_exact
6055,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2020-06-18,last14days,-d1-sailpoint+gartner,0,2,6_2020,sailpoint+gartner,sailpoint+gartner,,,-d1-sailpoint+gartner,,en,_en_sailpoint+gartner_sailpoint+gartner
6056,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2020-06-18,last14days,-d1-sailpoint+gartner,0,2,6_2020,sailpoint+gartner,sailpoint+gartner,,,-d1-sailpoint+gartner,,en,_en_sailpoint+gartner_sailpoint+gartner


In [756]:
NoPassNine['Topic'] = NoPassNine.apply(lambda x: 'events' if x['Keyword'] == 'microsoft' else x['Topic'],axis=1)
NoPassNine['Keyword'] = NoPassNine.apply(lambda x: 'microsoft-ignite-2020' if 'ignite' in x['Placeholder'] else x['Topic'],axis=1)
NoPassNine['Brand_x'] = NoPassNine.apply(lambda x: 'tm' if 'tm' in x['Placeholder'] else ('nb' if 'nb' in x['Placeholder'] else x['Brand_x']),axis=1)
NoPassNine['TargetingStrategy_x'] = NoPassNine.apply(lambda x: 'exact' if 'exact' in x['Placeholder'] else ('phrase' if 'phrase' in x['Placeholder'] else ('bmm' if 'bmm' in x['Placeholder'] else x['TargetingStrategy_x'])), axis=1)
NoPassNine['Keyword'] = NoPassNine.apply(lambda x: x['Placeholder'].split('_')[1] if '_sailpoint' in x['Placeholder'] else x['Keyword'],axis=1)
NoPassNine['Brand_x'] = NoPassNine['Brand_x'].apply(lambda x: x.replace('tm','br'))
NoPassNine['CleanCampaign'] = NoPassNine['Geo_x']+'_'+NoPassNine['Language_x']+'_'+NoPassNine['Brand_x']+'_'+NoPassNine['TargetingStrategy_x']
                                          

In [757]:
NoPassNine

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand_x,TargetingStrategy_x,Topic,Catchall_x,Keyword,Geo_x,Language_x,CleanCampaign
0,AUSTRALIANENERGYMARKETOPERATOR,Australia,aemo.com.au,https://www.sailpoint.com/identity-library/the-ultimate-guide-to-identity-governance/?elqct=paidmedia&elqchannel=googlesearch_apac-nb-exact-tm_sailpoint&gclid=eaiaiqobchmij7sxgmiv3givw7wwch0axabfeaayasaaegkrepd_bwe&elqcta=popup,2019-05-08,last14days,apac-nb-exact-tm_sailpoint,1,3,5_2019,br,exact,,,sailpoint,apac,en,apac_en_br_exact
1,AUSTRALIANENERGYMARKETOPERATOR,Australia,aemo.com.au,https://www.sailpoint.com/identity-library/the-ultimate-guide-to-identity-governance/?elqct=paidmedia&elqchannel=googlesearch_apac-nb-exact-tm_sailpoint&gclid=eaiaiqobchmij7sxgmiv3givw7wwch0axabfeaayasaaegkrepd_bwe&elqcta=popup,2019-05-02,last14days,apac-nb-exact-tm_sailpoint,1,3,5_2019,br,exact,,,sailpoint,apac,en,apac_en_br_exact
2,AUSTRALIANENERGYMARKETOPERATOR,Australia,aemo.com.au,https://www.sailpoint.com/partners/become-partner/?elqct=paidmedia&elqchannel=googlesearch_apac-nb-exact-tm_sailpoint&gclid=eaiaiqobchmij7sxgmiv3givw7wwch0axabfeaayasaaegkrepd_bwe,2019-05-02,last14days,apac-nb-exact-tm_sailpoint,1,3,5_2019,br,exact,,,sailpoint,apac,en,apac_en_br_exact
3,AUSTRALIANENERGYMARKETOPERATOR,Australia,aemo.com.au,https://www.sailpoint.com/partners/become-partner/?elqct=paidmedia&elqchannel=googlesearch_apac-nb-exact-tm_sailpoint&gclid=eaiaiqobchmij7sxgmiv3givw7wwch0axabfeaayasaaegkrepd_bwe,2019-05-07,last14days,apac-nb-exact-tm_sailpoint,1,3,5_2019,br,exact,,,sailpoint,apac,en,apac_en_br_exact
4,AUSTRALIANENERGYMARKETOPERATOR,Australia,aemo.com.au,https://www.sailpoint.com/identity-library/the-ultimate-guide-to-identity-governance/?elqct=paidmedia&elqchannel=googlesearch_apac-nb-exact-tm_sailpoint&gclid=eaiaiqobchmij7sxgmiv3givw7wwch0axabfeaayasaaegkrepd_bwe&elqcta=popup,2019-05-07,last14days,apac-nb-exact-tm_sailpoint,1,3,5_2019,br,exact,,,sailpoint,apac,en,apac_en_br_exact
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6053,H3C,People's Republic of China,h3c.com,https://www.sailpoint.com/events/digital-events-hub/?elqct=paidmedia&elqchannel=googlesearch_apac_en_nb_exact_events_microsoft_ignite_2020&gclid=eaiaiqobchmixcs_7eac6givk6ywch3_manfeaayasaaegiojvd_bwe,2020-06-15,last14days,apac_en_nb_exact_events_microsoft_ignite_2020,7,0,6_2020,nb,exact,events,,microsoft-ignite-2020,apac,en,apac_en_nb_exact
6054,H3C,People's Republic of China,h3c.com,https://www.sailpoint.com/events/digital-events-hub/?elqct=paidmedia&elqchannel=googlesearch_apac_en_nb_exact_events_microsoft_ignite_2020&gclid=eaiaiqobchmixcs_7eac6givk6ywch3_manfeaayasaaegiojvd_bwe,2020-06-15,last14days,apac_en_nb_exact_events_microsoft_ignite_2020,7,0,6_2020,nb,exact,events,,microsoft-ignite-2020,apac,en,apac_en_nb_exact
6055,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2020-06-18,last14days,-d1-sailpoint+gartner,0,2,6_2020,sailpoint+gartner,sailpoint+gartner,,,,,en,_en_sailpoint+gartner_sailpoint+gartner
6056,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2020-06-18,last14days,-d1-sailpoint+gartner,0,2,6_2020,sailpoint+gartner,sailpoint+gartner,,,,,en,_en_sailpoint+gartner_sailpoint+gartner


In [758]:
PassNineJoin = NoPassNine.merge(adwords,how="left",left_on=["CleanCampaign",'Topic','Keyword'], right_on=['CleanCampaign','Topic','Query'], indicator=True)
BothPassTen = PassNineJoin[PassNineJoin['_merge'] == 'both'].reset_index(drop=True)
NoPassTen = PassNineJoin[PassNineJoin['_merge'] != 'both'].reset_index(drop=True)
NoPassTen.drop(NoPassTen.columns[18:],axis=1,inplace=True)
len(BothPassTen)

39

In [769]:
NoPassTen

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand_x,TargetingStrategy_x,Topic,Catchall_x,Keyword,Geo_x,Language_x,CleanCampaign
0,SOUTHASIANTECH,Sri Lanka,satl.biz,https://www.sailpoint.com/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2019-05-21,last14days,-d1-sailpoint+gartner,0,2,5_2019,sailpoint+gartner,sailpoint+gartner,,,,,en,_en_sailpoint+gartner_sailpoint+gartner
1,SOUTHASIANTECH,Sri Lanka,satl.biz,https://www.sailpoint.com/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2019-05-21,last14days,-d1-sailpoint+gartner,0,2,5_2019,sailpoint+gartner,sailpoint+gartner,,,,,en,_en_sailpoint+gartner_sailpoint+gartner
2,SOUTHASIANTECH,Sri Lanka,satl.biz,https://www.sailpoint.com/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2019-05-21,last14days,-d1-sailpoint+gartner,0,2,5_2019,sailpoint+gartner,sailpoint+gartner,,,,,en,_en_sailpoint+gartner_sailpoint+gartner
3,SOUTHASIANTECH,Sri Lanka,satl.biz,https://www.sailpoint.com/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2019-05-21,last14days,-d1-sailpoint+gartner,0,2,5_2019,sailpoint+gartner,sailpoint+gartner,,,,,en,_en_sailpoint+gartner_sailpoint+gartner
4,SAILPOINTTECH,United States,sailpoint.com,https://www.sailpoint.com/identity-library/identity-and-access-management/?elqct=paidmedia&elqchannel=googlesearch_apac-nb-bmm-id-gov_identity-access-management&gclid=cj0kcqiaxzpgbrcmarisaorthsaccjqj8p1pukwfx2lomdeaby0pehm7yka_dsdfgyldkbtgxdjed04aarqyealw_wcb,2019-05-31,last14days,apac-nb-bmm-id-gov_identity-access-management,1,6,5_2019,nb,bmm,,gov_identity-access-management,identity-access-management,apac,en,apac_en_nb_bmm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6014,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2020-06-18,last14days,-d1-sailpoint+gartner,0,2,6_2020,sailpoint+gartner,sailpoint+gartner,,,,,en,_en_sailpoint+gartner_sailpoint+gartner
6015,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2020-06-18,last14days,-d1-sailpoint+gartner,0,2,6_2020,sailpoint+gartner,sailpoint+gartner,,,,,en,_en_sailpoint+gartner_sailpoint+gartner
6016,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2020-06-18,last14days,-d1-sailpoint+gartner,0,2,6_2020,sailpoint+gartner,sailpoint+gartner,,,,,en,_en_sailpoint+gartner_sailpoint+gartner
6017,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2020-06-18,last14days,-d1-sailpoint+gartner,0,2,6_2020,sailpoint+gartner,sailpoint+gartner,,,,,en,_en_sailpoint+gartner_sailpoint+gartner


In [780]:
NoPassTen['Keyword'] = NoPassTen.apply(lambda x: x['Placeholder'].split('_')[1] if len(x['Placeholder'].split('_')) > 1 else x['Keyword'],axis=1)
NoPassTen['Catchall_x'] = NoPassTen.apply(lambda x: x['Placeholder'].split('_')[0] if len(x['Placeholder'].split('_')) > 1 else x['Keyword'],axis=1)
NoPassTen['Topic'] = NoPassTen.apply(lambda x: 'dag' if '-dag' in x['Catchall_x'] else ('id-gov' if 'id-gov' in x['Catchall_x'] else x['Topic']), axis=1)
NoPassTen['CleanCampaign']=NoPassTen['CleanCampaign'].apply(lambda x: x.replace('_br_','_nb_'))
NoPassTen.sample(10)

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand_x,TargetingStrategy_x,Topic,Catchall_x,Keyword,Geo_x,Language_x,CleanCampaign
976,GRATTAN,United Kingdom,fgh-uk.com,https://www.sailpoint.com/identity-library/governing-access-to-data-stored-in-files/?elqct=paidmedia&elqchannel=googlesearch_int-tm-phrase-dag_varonis&gclid=eaiaiqobchmi9kwo5js55aivbldtch1m-a1ueaayaiaaegl1rfd_bwe,2019-09-05,last14days,int-tm-phrase-dag_varonis,1,3,9_2019,br,phrase,dag,int-tm-phrase-dag,varonis,int,en,int_en_nb_phrase
3494,,United States,jcp.com,https://www.sailpoint.com/identity-library/getting-compliant-with-identity-governance-for-files/?elqct=paidmedia&elqchannel=googlesearch_usa-tm-exact-dag_varonis&gclid=cj0kcqjw5rbsbrcfarisageyrwdomxogncujlkjbdm-x-flonhvf4nw2-rmwe9lrypgm-oskfjvzulgaatwwealw_wcb,2019-09-28,last14days,usa-tm-exact-dag_varonis,1,3,9_2019,br,exact,dag,usa-tm-exact-dag,varonis,usa,en,usa_en_nb_exact
508,COCACOLAAMATIL,Australia,spc.com.au,https://www.sailpoint.com/identity-library/getting-compliant-with-identity-governance-for-files/?elqct=paidmedia&elqchannel=googlesearch_apac-tm-phrase-dag_data-access-governance&gclid=eaiaiqobchmiqm-3vssn5aivxirwch0wqwgxeaayasaaegl5ovd_bwe,2019-08-29,last14days,apac-tm-phrase-dag_data-access-governance,1,5,8_2019,br,phrase,dag,apac-tm-phrase-dag,data-access-governance,apac,en,apac_en_nb_phrase
4506,,United Kingdom,accenture.com,https://www.sailpoint.com/identity-library/getting-compliant-with-identity-governance-for-files/?elqct=paidmedia&elqchannel=googlesearch_int-tm-bmm-dag_data-classification&gclid=eaiaiqobchmi8d3k7-pr5aivueptch04nahreaayasaaegjm6pd_bwe,2019-09-25,last14days,int-tm-bmm-dag_data-classification,1,4,9_2019,br,bmm,dag,int-tm-bmm-dag,data-classification,int,en,int_en_nb_bmm
3635,VARONISSYSTEMS,United States,varonis.com,https://www.sailpoint.com/identity-library/getting-compliant-with-identity-governance-for-files/?elqct=paidmedia&elqchannel=googlesearch_usa-tm-phrase-dag_data-access-governance&gclid=cj0kcqjwiilsbrcgarisahkqwlp9yspjzhzqnmb1ysgdy3f-ahkuoganws622ltmghnwek4mqh_-nwiaalpfealw_wcb,2019-09-17,last14days,usa-tm-phrase-dag_data-access-governance,1,5,9_2019,br,phrase,dag,usa-tm-phrase-dag,data-access-governance,usa,en,usa_en_nb_phrase
5454,CUNY,United States,cuny.edu,https://www.sailpoint.com/identity-library/getting-compliant-with-identity-governance-for-files/?elqct=paidmedia&elqchannel=googlesearch_usa-tm-phrase-dag_data-classification&gclid=cj0kcqjwivbsbrdsarisadyisj8ghgdtltg1qnvxddzualz2vlzya523ylpc1fkxivxblfmnwpwib2uaavfzealw_wcb,2019-10-09,last14days,usa-tm-phrase-dag_data-classification,1,4,10_2019,br,phrase,dag,usa-tm-phrase-dag,data-classification,usa,en,usa_en_nb_phrase
1140,DIALOGINFORMATION,Australia,rustreport.com.au,https://www.sailpoint.com/identity-library/getting-compliant-with-identity-governance-for-files/?elqct=paidmedia&elqchannel=googlesearch_apac-tm-phrase-dag_data-security&gclid=cj0kcqjw5mlrbrclarisapg0wgywkgmtqcahnvxrs8rtvccogxwfmdcnbnqsuzbszy_4wbkoogoi1zcaaigvealw_wcb,2019-09-05,last14days,apac-tm-phrase-dag_data-security,1,4,9_2019,br,phrase,dag,apac-tm-phrase-dag,data-security,apac,en,apac_en_nb_phrase
470,,South Africa,mtn.com,https://www.sailpoint.com/identity-library/protecting-intellectual-property-data-access-governance/?elqct=paidmedia&elqchannel=googlesearch_int-tm-phrase-dag_dag&gclid=eaiaiqobchmi5vwv4lg65aivbrttch1l8aiyeaayasaaegisgvd_bwe,2019-09-05,last14days,int-tm-phrase-dag_dag,1,3,9_2019,br,phrase,dag,int-tm-phrase-dag,dag,int,en,int_en_nb_phrase
2429,,Isle of Man,manx.net,https://www.sailpoint.com/identity-library/getting-compliant-with-identity-governance-for-files/?elqct=paidmedia&elqchannel=googlesearch_int-tm-phrase-dag_varonis&gclid=cjwkcajw5fzrbraseiwad2osvxrlvbny-mxfigyyds8vd48rzchv8ajhuzpu_8pzqrymhyasfcf6-xocfikqavd_bwe,2019-09-16,last14days,int-tm-phrase-dag_varonis,1,3,9_2019,br,phrase,dag,int-tm-phrase-dag,varonis,int,en,int_en_nb_phrase
3328,MEDICALSCHOOLSCOUNCIL,United Kingdom

In [784]:
PassTenJoin = NoPassTen.merge(adwords,how="left",left_on=["CleanCampaign",'Topic','Keyword'], right_on=['CleanCampaign','Topic','Query'], indicator=True)
BothPassEleven = PassTenJoin[PassTenJoin['_merge'] == 'both'].reset_index(drop=True)
NoPassEleven = PassTenJoin[PassTenJoin['_merge'] != 'both'].reset_index(drop=True)
#NoPassEleven.drop(NoPassEleven.columns[18:],axis=1,inplace=True)
len(BothPassEleven)

5777

In [785]:
NoPassEleven

,name,country,domain,url,visiteddate,DateRange,Placeholder,Underscores,Hyphens,month,Brand_x,TargetingStrategy_x,Topic,Catchall_x,Keyword,Geo_x,Language_x,CleanCampaign,Search keyword,Search keyword status,Search keyword match type,Campaign,Campaign status,Ad group,Keyword max CPC,Currency,Keyword/Placement,Start date,End date,Clicks,Impressions,CTR,Avg. CPC,Cost,Impr. (Abs. Top) %,Impr. (Top) %,Conversions,View-through conv.,Cost / conv.,Conv. rate,Avg. CPM,Query,Geo,Language,Brand,TargetingStrategy,Catchall,_merge
0,SOUTHASIANTECH,Sri Lanka,satl.biz,https://www.sailpoint.com/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2019-05-21,last14days,-d1-sailpoint+gartner,0,2,5_2019,sailpoint+gartner,sailpoint+gartner,,,,,en,_en_sailpoint+gartner_sailpoint+gartner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,SOUTHASIANTECH,Sri Lanka,satl.biz,https://www.sailpoint.com/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2019-05-21,last14days,-d1-sailpoint+gartner,0,2,5_2019,sailpoint+gartner,sailpoint+gartner,,,,,en,_en_sailpoint+gartner_sailpoint+gartner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,SOUTHASIANTECH,Sri Lanka,satl.biz,https://www.sailpoint.com/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2019-05-21,last14days,-d1-sailpoint+gartner,0,2,5_2019,sailpoint+gartner,sailpoint+gartner,,,,,en,_en_sailpoint+gartner_sailpoint+gartner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,SOUTHASIANTECH,Sri Lanka,satl.biz,https://www.sailpoint.com/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2019-05-21,last14days,-d1-sailpoint+gartner,0,2,5_2019,sailpoint+gartner,sailpoint+gartner,,,,,en,_en_sailpoint+gartner_sailpoint+gartner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,SAILPOINTTECH,United States,sailpoint.com,https://www.sailpoint.com/identity-library/identity-and-access-management/?elqct=paidmedia&elqchannel=googlesearch_apac-nb-bmm-id-gov_identity-access-management&gclid=cj0kcqiaxzpgbrcmarisaorthsaccjqj8p1pukwfx2lomdeaby0pehm7yka_dsdfgyldkbtgxdjed04aarqyealw_wcb,2019-05-31,last14days,apac-nb-bmm-id-gov_identity-access-management,1,6,5_2019,nb,bmm,id-gov,apac-nb-bmm-id-gov,identity-access-management,apac,en,apac_en_nb_bmm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2020-06-18,last14days,-d1-sailpoint+gartner,0,2,6_2020,sailpoint+gartner,sailpoint+gartner,,,,,en,_en_sailpoint+gartner_sailpoint+gartner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
238,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2020-06-18,last14days,-d1-sailpoint+gartner,0,2,6_2020,sailpoint+gartner,sailpoint+gartner,,,,,en,_en_sail

In [775]:
adwords[ (adwords['Query'].str.contains('data-security'))]

,Search keyword,Search keyword status,Search keyword match type,Campaign,Campaign status,Ad group,Keyword max CPC,Currency,Keyword/Placement,Start date,End date,Clicks,Impressions,CTR,Avg. CPC,Cost,Impr. (Abs. Top) %,Impr. (Top) %,Conversions,View-through conv.,Cost / conv.,Conv. rate,Avg. CPM,Query,Geo,Language,Brand,TargetingStrategy,Topic,Catchall,CleanCampaign
565,data security,Campaign paused,Phrase,sem_int_en_nb_phrase_dag,Campaign paused,data-security,7.61,USD,data security,2018-10-15,--,488,"23,174",2.11%,10.05,"4,904.94",31.22%,74.93%,3.0,0,"1,634.98",0.61%,211.66,data-security,int,en,nb,phrase,dag,,int_en_nb_phrase
589,+data +security,Campaign paused,Broad,sem_usa_en_nb_bmm_dag,Campaign paused,data-security,5.76,USD,+data +security,2018-08-26,--,211,"23,646",0.89%,7.28,"1,536.47",9.60%,43.36%,2.0,0,768.24,0.95%,64.98,data-security,usa,en,nb,bmm,dag,,usa_en_nb_bmm
638,data security laws,Campaign paused,Phrase,sem_usa_en_nb_phrase_ccpa,Campaign paused,data-security-laws,8.00,USD,data security laws,2019-07-29,--,3,558,0.54%,9.94,29.82,12.37%,30.29%,0.0,0,0.00,0.00%,53.44,data-security-laws,usa,en,nb,phrase,ccpa,,usa_en_nb_phrase
714,data security regulations,Campaign paused,Phrase,sem_apac_en_nb_phrase_gdpr,Campaign paused,data-security-regulations,7.50,USD,data security regulations,2018-10-24,--,0,16,0.00%,0.00,0.00,43.75%,81.25%,0.0,0,0.00,0.00%,0.00,data-security-regulations,apac,en,nb,phrase,gdpr,,apac_en_nb_phrase
718,data security regulations,Campaign paused,Phrase,sem_ww_en_nb_rtg_phrase_gdpr,Campaign paused,data-security-regulations,8.00,USD,data security regulations,2020-02-10,--,0,1,0.00%,0.00,0.00,0.00%,0.00%,0.0,0,0.00,0.00%,0.00,data-security-regulations,ww,en,nb,rtg,phrase,[gdpr],ww_en_nb_rtg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17896,data security policies,Campaign paused,Broad,data access governance - search,Campaign paused,ad group #1,13.33,USD,data security policies,2016-11-23,--,3,819,0.37%,7.55,22.65,0.00%,0.00%,0.0,0,0.00,0.00%,27.66,data-security-policies,,,,,data access governance - search,,___
17899,data security protection,Campaign paused,Broad,data access governance - broad,Campaign paused,data governance broad,20.00,USD,data security protection,2016-05-11,--,1,62,1.61%,4.93,4.93,0.00%,0.00%,0.0,0,0.00,0.00%,79.52,data-security-protection,,,,,data access governance - broad,,___
18083,data security compliance standards,Campaign paused,Broad,data access governance - broad,Campaign paused,data governance broad,41.80,USD,data security compliance standards,2016-05-11,--,6,666,0.90%,17.40,104.39,0.00%,0.00%,0.0,0,0.00,0.00%,156.74,data-security-compliance-standards,,,,,data access governance - broad,,___
18776,box data security,Campaign paused,Broad,data access governance - broad,Campaign paused,data governance broad whitepaper,20.00,USD,box data security,2016-05-11,--,1,387,0.26%,8.46,8.46,0.00%,0.00%,0.0,0,0.00,0.00%,21.86,box-data-security,,,,,data access governance - broad,,___


In [786]:
dfs = [BothFirstPass,SiteLink,NonEnglish,BothSecondPass,DroppedDash,BothPassThree,BothPassFour,BothPassFive,BothPassSix,BothPassSeven,BothPassEight,BothPassNine,BothPassTen,BothPassEleven,NoPassEleven]
FINAL=pd.concat(dfs)

In [800]:
1-((len(FINAL[FINAL['Campaign'].isnull()])-len(SiteLink)-len(NonEnglish))/len(FINAL))

0.9993999637994689

In [787]:
FINAL.to_csv('Final_Matching.csv')

In [811]:
FINAL=FINAL.drop(['Underscores','Hyphens','Geo_x','Language_x','TargetingStrategy_x','Brand_x','Catchall_x','Placeholder','Topic','Keyword','Geo_y','Language_y','TargetingStrategy_y','Catchall_y','_merge','Geo','Language','Brand','TargetingStrategy','Catchall','Query','Brand_y'],axis=1)

KeyError: "['Underscores' 'Hyphens' 'Geo_x' 'Language_x' 'TargetingStrategy_x'\n 'Brand_x' 'Catchall_x' 'Placeholder' 'Topic' 'Keyword' 'Geo_y'\n 'Language_y' 'TargetingStrategy_y' 'Catchall_y' '_merge' 'Geo' 'Language'\n 'Brand' 'TargetingStrategy' 'Catchall' 'Query' 'Brand_y'] not found in axis"

In [810]:
FINAL=FINAL.drop([],axis=1)
FINAL

,name,country,domain,url,visiteddate,DateRange,month,CleanCampaign,Search keyword,Search keyword status,Search keyword match type,Campaign,Campaign status,Ad group,Keyword max CPC,Currency,Keyword/Placement,Start date,End date,Clicks,Impressions,CTR,Avg. CPC,Cost,Impr. (Abs. Top) %,Impr. (Top) %,Conversions,View-through conv.,Cost / conv.,Conv. rate,Avg. CPM
0,IBEX,Philippines,dtsigroup.com,https://www.sailpoint.com/identity-library/identity-and-access-management/?elqct=paidmedia&elqchannel=googlesearch_apac-en-nb-exact-id-gov_identity-access-management&gclid=eaiaiqobchmi6ff9vctp4qivlsqrch36uqg-eaayasaaegl0x_d_bwe,2019-04-14,last14days,4_2019,apac_en_nb_exact,identity access management,Campaign paused,Exact,sem_apac_en_nb_exact_id-gov,Campaign paused,identity-and-access-management,7.50,USD,identity access management,2018-10-24,--,84,"2,435",3.45%,9.79,822.09,30.43%,90.97%,2.0,0.0,411.04,2.38%,337.61
1,IBEX,Philippines,dtsigroup.com,https://www.sailpoint.com/identity-management-platform/?elqct=paidmedia&elqchannel=googlesearch_apac-en-nb-exact-id-gov_identity-access-management&gclid=eaiaiqobchmi6ff9vctp4qivlsqrch36uqg-eaayasaaegl0x_d_bwe,2019-04-14,last14days,4_2019,apac_en_nb_exact,identity access management,Campaign paused,Exact,sem_apac_en_nb_exact_id-gov,Campaign paused,identity-and-access-management,7.50,USD,identity access management,2018-10-24,--,84,"2,435",3.45%,9.79,822.09,30.43%,90.97%,2.0,0.0,411.04,2.38%,337.61
2,IBEX,Philippines,dtsigroup.com,https://www.sailpoint.com/identity-library/identity-and-access-management/?elqct=paidmedia&elqchannel=googlesearch_apac-en-nb-exact-id-gov_identity-access-management&gclid=eaiaiqobchmi6ff9vctp4qivlsqrch36uqg-eaayasaaegl0x_d_bwe,2019-04-14,last14days,4_2019,apac_en_nb_exact,identity access management,Campaign paused,Exact,sem_apac_en_nb_exact_id-gov,Campaign paused,identity-and-access-management,7.50,USD,identity access management,2018-10-24,--,84,"2,435",3.45%,9.79,822.09,30.43%,90.97%,2.0,0.0,411.04,2.38%,337.61
3,BRIDGENET SOLUTIONS SDN BHD,Malaysia,bridgenet.com.my,https://www.sailpoint.com/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch_apac-en-nb-phrase-gartner_identity-management-magic-quadrant&gclid=eaiaiqobchmiu6rl0psz4qivsiipch0waqrqeaayaiaaegkal_d_bwe,2019-04-03,last14days,4_2019,apac_en_nb_phrase,identity management magic quadrant,Campaign paused,Phrase,sem_apac_en_nb_phrase_gartner,Campaign paused,gartner-identity-management,6.75,USD,identity management magic quadrant,2018-10-24,--,4,48,8.33%,6.43,25.73,47.92%,91.67%,0.0,0.0,0.00,0.00%,536.04
4,BRIDGENET SOLUTIONS SDN BHD,Malaysia,bridgenet.com.my,https://www.sailpoint.com/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch_apac-en-nb-phrase-gartner_identity-management-magic-quadrant&gclid=eaiaiqobchmiu6rl0psz4qivsiipch0waqrqeaayaiaaegkal_d_bwe,2019-04-03,last14days,4_2019,apac_en_nb_phrase,identity management magic quadrant,Campaign paused,Phrase,sem_apac_en_nb_phrase_gartner,Campaign paused,gartner-identity-management,6.75,USD,identity management magic quadrant,2018-10-24,--,4,48,8.33%,6.43,25.73,47.92%,91.67%,0.0,0.0,0.00,0.00%,536.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2020-06-18,last14days,6_2020,_en_sailpoint+gartner_sailpoint+gartner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,ASML,Netherlands,asml.com,https://www.sailpoint.com/identity-library/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch-d1-sailpoint+gartner&gclid=cii6zqpcxticfuudvaodmzyhuw,2020-06-18,last14days,6_2020,_en_sailpoint+gartner_sailpoint+gartner,NaN,NaN,NaN,NaN,Na

In [801]:
### Bring in TAL
SFDC = pd.read_csv('TAL SFDC List.csv')
SFDC['Website']=SFDC['Website'].str.lstrip('http://')
SFDC['Website']=SFDC['Website'].str.lstrip('https://')
SFDC['Website']=SFDC['Website'].str.lstrip('www.')
SFDC['Website']=SFDC['Website'].astype('str').str.lower()

In [802]:
SFDC.sample(10)

,Parent Account ID,Column1,Parent Account,Account ID,Account Name,SailPoint BOAT (OLD),6sense Account Buying Stage – Sailpoint,Reporting Total Leads and Contacts,Account Owner,Current Target Account,Industry,Employee Range,Employees,Type,Account Record Type,Website,6sense Account Intent Score-Sailpoint,6sense Account Profile Fit – Sailpoint,6sense Target Lists,6sense Modified,Last Activity,SailPoint BOAT,Sales Territory Rollup,Billing GEO,Billing Country,Billing State/Province,Billing City,Billing Zip/Postal Code,6sense Target List- Researching,6sense Target List- Tech Used
14942,000000000000000,NaN,NaN,0012J00002NQR6v,OneDigital Health & Benefits,NaN,Consideration,NaN,Matt Slagter,True,Insurance,NaN,2500.0,Prospect,Standard Account,onedigital.com,64.0,Strong,NaN,NaN,NaN,SR-AMS-SE,AMS-Southeast,AMS,US,GA,Atlanta,30339,NaN,NaN
8857,000000000000000,NaN,NaN,0015000001IVV4l,Red Electrica Española,NaN,Awareness,NaN,Jorge Sendra Mas,True,Energy,NaN,2050.0,Prospect,Standard Account,ree.es/en,35.0,Strong,NaN,NaN,8/26/2020,SR-EMEA-South_2,EMEA-South,EMEA,Spain,Comunidad de Madrid,Alcobendas,28109,NaN,",,052,045"
5,0015000000tEUai,NaN,KPN-Managed Service Provider,0015000000RRqZR,UWV,SR-EMEA-Benelux,Consideration,NaN,Gabor Tas,True,Insurance,10k+,16672.0,Expired,Standard Account,uwv.nl,67.0,Strong,",091_2_1,112_2_1,091_2,091,020,084,084_2,084_2_1,107_2_1",NaN,3/23/2020,SR-EMEA-Benelux,EMEA-Benelux,EMEA,Netherlands,Nh,Amsterdam,1040 Hg,NaN,",080,052, 020"
15682,000000000000000,NaN,NaN,0015000000eO5rR,The Toronto-Dominion Bank,SR-AMS-NECA,Purchase,NaN,Karen Escoffery,True,Banking,10k+,89031.0,Customer,Standard Customer Account,tdbank.com,98.0,Strong,",020,030,057_1,072,072_1,052,091,091_1,112_1,115_1,110_1",NaN,8/25/2020,SR-AMS-CAN,AMS-Northeast,AMS,Canada,On,Toronto,M5K 1A2,",066,115,",",,,035,033,036,038,037,,,060,070,071,052,045,021,073,022"
13556,000000000000000,NaN,NaN,0015000000tEUcF,"Metro, Inc.",SR-AMS-NECA,Consideration,NaN,Karen Escoffery,True,Retail,NaN,150000.0,Prospect,Standard Account,metroag.de/en,64.0,Strong,",057_1,065_1,065_1_1,074,074_1,085,085_1,108_1,089,089_1,091,091_1,112_1,084,084_1,107_1",NaN,8/25/2020,SR-AMS-CAN,AMS-Northeast,AMS,Canada,Quebec,Tampa,336042626,NaN,",,045,021,073,080,100,"
16025,000000000000000,NaN,NaN,0015000000mPZKO,Vodafone Group (Czech)PLC,NaN,Awareness,NaN,Jakub Orkiszewski,True,Retail,NaN,0.0,Prospect,Standard Account,vodafone.com/content/index.html,37.0,Strong,",114_2_1,111_2_1,110_2_1",NaN,NaN,SR-EMEA-Nordics,EMEA-Nordics,EMEA,Czech Republic,NaN,Hodonín,69501,NaN,",113,,024,,025,,022,"
1367,0015000000wMch9,NaN,U.S. Department of Energy,00150000014yBLq,Lawrence Livermore National Laboratory - LLNL,SR-AMS-FED,Decision,NaN,Paul Cimino,True,Federal Government,NaN,6586.0,Prospect,Standard Account,llnl.gov,70.0,Strong,",083,064,091_1,112_1,083_1,065_1,068_1,065_1_1,074_1,085_1,108_1,089,060,091,068,064_1,074,057_1,089_1,085,084,084_1,107_1093_1,105_1,098_1,104_1,097_1,109_1,115_1",NaN,6/3/2020,SR-AMS-FED,AMS-Federal,AMS,United States,California,Livermore,94550,",066,115,",",033,083,064,083_1,065_1,068_1,065_1_1,074_1,080,085_1,089,060,068,064_1,074,052, 057_1,089_1,085"
11196,000000000000000,NaN,NaN,0012J00002KSchr,Schuler,SR-EMEA-DACH,Awareness,NaN,Christiane Spranz,True,Other,5k-10k,5464.0,Prospect,Standard Account,schulergroup.com,31.0,Moderate,",015,075,075_2,085,085_2,108_2_3,087,087_3,090,090_2,091,091_2,091_2_3,112_2_3,117",NaN,6/17/2020,SR-EMEA-DACH,EMEA-DACH,EMEA,Germany,Bw,Goppingen,73033,NaN,",045"
19776,000000000000000,NaN,NaN,0012J00002J5xiX,Fluvius System Operator CV,SR-EMEA-Benelux,Awareness,NaN,Ton Luyk,True,Utilities,5k-10k,5077.0,Prospect,Standard Account,infrax.be,36.0,Strong,",074,085_2,108_2_1,089,084,084_2_1,107_2_1,074_2,084_2,091_2_1,112_2_1,091_2,082,082_2,091,085,089_2",NaN,8/25/2020,SR-EMEA-Benelux,EMEA-Benelux,EMEA,Belgium,Limburg,Melle,9090,NaN,",080,074,033,052,085_2,089,084,084_2_1,074_2,084_2,082,082_2,085,089_2"
1416,0015000000uOo86,NaN,AMC Entertainment,0015000001

In [812]:
FINAL_TAL = FINAL.merge(SFDC,how="left",left_on='domain', right_on='Website')
#FINAL_TAL = SUPERJOIN_6SENSE_TAL.drop_duplicates()
print("Superjoin TALs merged")

Superjoin TALs merged


In [813]:
FINAL_TAL


,name,country,domain,url,visiteddate,DateRange,month,CleanCampaign,Search keyword,Search keyword status,Search keyword match type,Campaign,Campaign status,Ad group,Keyword max CPC,Currency,Keyword/Placement,Start date,End date,Clicks,Impressions,CTR,Avg. CPC,Cost,Impr. (Abs. Top) %,Impr. (Top) %,Conversions,View-through conv.,Cost / conv.,Conv. rate,Avg. CPM,Parent Account ID,Column1,Parent Account,Account ID,Account Name,SailPoint BOAT (OLD),6sense Account Buying Stage – Sailpoint,Reporting Total Leads and Contacts,Account Owner,Current Target Account,Industry,Employee Range,Employees,Type,Account Record Type,Website,6sense Account Intent Score-Sailpoint,6sense Account Profile Fit – Sailpoint,6sense Target Lists,6sense Modified,Last Activity,SailPoint BOAT,Sales Territory Rollup,Billing GEO,Billing Country,Billing State/Province,Billing City,Billing Zip/Postal Code,6sense Target List- Researching,6sense Target List- Tech Used
0,IBEX,Philippines,dtsigroup.com,https://www.sailpoint.com/identity-library/identity-and-access-management/?elqct=paidmedia&elqchannel=googlesearch_apac-en-nb-exact-id-gov_identity-access-management&gclid=eaiaiqobchmi6ff9vctp4qivlsqrch36uqg-eaayasaaegl0x_d_bwe,2019-04-14,last14days,4_2019,apac_en_nb_exact,identity access management,Campaign paused,Exact,sem_apac_en_nb_exact_id-gov,Campaign paused,identity-and-access-management,7.50,USD,identity access management,2018-10-24,--,84,"2,435",3.45%,9.79,822.09,30.43%,90.97%,2.0,0.0,411.04,2.38%,337.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IBEX,Philippines,dtsigroup.com,https://www.sailpoint.com/identity-management-platform/?elqct=paidmedia&elqchannel=googlesearch_apac-en-nb-exact-id-gov_identity-access-management&gclid=eaiaiqobchmi6ff9vctp4qivlsqrch36uqg-eaayasaaegl0x_d_bwe,2019-04-14,last14days,4_2019,apac_en_nb_exact,identity access management,Campaign paused,Exact,sem_apac_en_nb_exact_id-gov,Campaign paused,identity-and-access-management,7.50,USD,identity access management,2018-10-24,--,84,"2,435",3.45%,9.79,822.09,30.43%,90.97%,2.0,0.0,411.04,2.38%,337.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IBEX,Philippines,dtsigroup.com,https://www.sailpoint.com/identity-library/identity-and-access-management/?elqct=paidmedia&elqchannel=googlesearch_apac-en-nb-exact-id-gov_identity-access-management&gclid=eaiaiqobchmi6ff9vctp4qivlsqrch36uqg-eaayasaaegl0x_d_bwe,2019-04-14,last14days,4_2019,apac_en_nb_exact,identity access management,Campaign paused,Exact,sem_apac_en_nb_exact_id-gov,Campaign paused,identity-and-access-management,7.50,USD,identity access management,2018-10-24,--,84,"2,435",3.45%,9.79,822.09,30.43%,90.97%,2.0,0.0,411.04,2.38%,337.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BRIDGENET SOLUTIONS SDN BHD,Malaysia,bridgenet.com.my,https://www.sailpoint.com/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch_apac-en-nb-phrase-gartner_identity-management-magic-quadrant&gclid=eaiaiqobchmiu6rl0psz4qivsiipch0waqrqeaayaiaaegkal_d_bwe,2019-04-03,last14days,4_2019,apac_en_nb_phrase,identity management magic quadrant,Campaign paused,Phrase,sem_apac_en_nb_phrase_gartner,Campaign paused,gartner-identity-management,6.75,USD,identity management magic quadrant,2018-10-24,--,4,48,8.33%,6.43,25.73,47.92%,91.67%,0.0,0.0,0.00,0.00%,536.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BRIDGENET SOLUTIONS SDN BHD,Malaysia,bridgenet.com.my,https://www.sailpoint.com/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch_apac-en-nb-phrase-gartner_identity-management-magic-quadrant&gclid=eaiaiqobchmiu6rl0psz4qivsiipch0waqrqeaayaiaaegkal_d_bwe,2019-04-03,last14days,4_2019,apac_en_nb_phrase,iden

In [814]:
len(FINAL_TAL)

502082

In [815]:
FINAL_TAL_2 = FINAL_TAL.drop_duplicates()

In [816]:
len(FINAL_TAL_2)

115460

In [821]:
FINAL_TAL_2.to_csv('Final_Adwords_With_TAL.csv',index=False)

In [819]:
FINAL_TAL_2['visiteddate'].describe

<bound method NDFrame.describe of 0         2019-04-14
1         2019-04-14
3         2019-04-03
5         2019-04-08
7         2019-04-08
             ...    
502068    2020-05-13
502070    2020-05-18
502071    2020-05-21
502076    2020-06-16
502077    2020-06-18
Name: visiteddate, Length: 115460, dtype: object>

In [820]:
FINAL_TAL_2


,name,country,domain,url,visiteddate,DateRange,month,CleanCampaign,Search keyword,Search keyword status,Search keyword match type,Campaign,Campaign status,Ad group,Keyword max CPC,Currency,Keyword/Placement,Start date,End date,Clicks,Impressions,CTR,Avg. CPC,Cost,Impr. (Abs. Top) %,Impr. (Top) %,Conversions,View-through conv.,Cost / conv.,Conv. rate,Avg. CPM,Parent Account ID,Column1,Parent Account,Account ID,Account Name,SailPoint BOAT (OLD),6sense Account Buying Stage – Sailpoint,Reporting Total Leads and Contacts,Account Owner,Current Target Account,Industry,Employee Range,Employees,Type,Account Record Type,Website,6sense Account Intent Score-Sailpoint,6sense Account Profile Fit – Sailpoint,6sense Target Lists,6sense Modified,Last Activity,SailPoint BOAT,Sales Territory Rollup,Billing GEO,Billing Country,Billing State/Province,Billing City,Billing Zip/Postal Code,6sense Target List- Researching,6sense Target List- Tech Used
0,IBEX,Philippines,dtsigroup.com,https://www.sailpoint.com/identity-library/identity-and-access-management/?elqct=paidmedia&elqchannel=googlesearch_apac-en-nb-exact-id-gov_identity-access-management&gclid=eaiaiqobchmi6ff9vctp4qivlsqrch36uqg-eaayasaaegl0x_d_bwe,2019-04-14,last14days,4_2019,apac_en_nb_exact,identity access management,Campaign paused,Exact,sem_apac_en_nb_exact_id-gov,Campaign paused,identity-and-access-management,7.50,USD,identity access management,2018-10-24,--,84,"2,435",3.45%,9.79,822.09,30.43%,90.97%,2.0,0.0,411.04,2.38%,337.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IBEX,Philippines,dtsigroup.com,https://www.sailpoint.com/identity-management-platform/?elqct=paidmedia&elqchannel=googlesearch_apac-en-nb-exact-id-gov_identity-access-management&gclid=eaiaiqobchmi6ff9vctp4qivlsqrch36uqg-eaayasaaegl0x_d_bwe,2019-04-14,last14days,4_2019,apac_en_nb_exact,identity access management,Campaign paused,Exact,sem_apac_en_nb_exact_id-gov,Campaign paused,identity-and-access-management,7.50,USD,identity access management,2018-10-24,--,84,"2,435",3.45%,9.79,822.09,30.43%,90.97%,2.0,0.0,411.04,2.38%,337.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BRIDGENET SOLUTIONS SDN BHD,Malaysia,bridgenet.com.my,https://www.sailpoint.com/identity-governance-leader-gartner-magic-quadrant/?elqct=paidmedia&elqchannel=googlesearch_apac-en-nb-phrase-gartner_identity-management-magic-quadrant&gclid=eaiaiqobchmiu6rl0psz4qivsiipch0waqrqeaayaiaaegkal_d_bwe,2019-04-03,last14days,4_2019,apac_en_nb_phrase,identity management magic quadrant,Campaign paused,Phrase,sem_apac_en_nb_phrase_gartner,Campaign paused,gartner-identity-management,6.75,USD,identity management magic quadrant,2018-10-24,--,4,48,8.33%,6.43,25.73,47.92%,91.67%,0.0,0.0,0.00,0.00%,536.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,UNITEDASSOCIATION,United States,uanet.org,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=eaiaiqobchmih93_scja4qivffmgch0mewqveaayasaaeglk9vd_bwe,2019-04-08,last14days,4_2019,usa_en_br_exact,sailpoint,Campaign paused,Exact,sem_usa_en_br_exact,Campaign paused,brand,2.13,USD,sailpoint,2018-05-06,--,"46,591","282,311",16.50%,1.58,"73,384.76",81.50%,99.65%,360.0,0.0,203.83,0.77%,259.94,000000000000000,NaN,NaN,0015000001R7gJw,United Association,SR-AMS-Mid-Atlantic,Awareness,NaN,Bob Caldiero,True,Non-Profit and Associations,NaN,3400.0,Prospect,Standard Account,uanet.org,37.0,Weak,",057_1,065_1,065_1_1,091,091_1,112_1,098_1,104_1",NaN,2/21/2020,SR-AMS-NY Metro & Mid-Atlantic,AMS-Mid-Atlantic,AMS,United States,Maryland,Annapolis,21401-3687,",066",NaN
7,UNITEDASSOCIATION,United States,uanet.org,https://www.sailpoint.com/?elqct=paidmedia&elqchannel=googlesearch_usa-en-br-exact_sailpoint&gclid=eaiaiqobchmi9rjqs8va4qivgiagch3kpgf_eaayasaaegiy1fd_bwe,2019-04-08,las

In [61]:
SFDC.sample(10)


NameError: name 'SFDC' is not defined

In [68]:
TAL6SenseLJoin = Sense6SearchLJoin.merge(SFDC,how='left',left_on="domain",right_on="Website")
TALAdwordsLJoin = adwordsLJoin.merge(SFDC,how='left',left_on="domain",right_on="Website")


In [69]:
TAL6SenseLJoin.to_csv('TAL6SenseLJoin.csv')
TALAdwordsLJoin.to_csv('TALAdwordsLJoin.csv')

In [ ]:
Sense6Search[(Sense6Search['Campaign'] == "") & (Sense6Search['url'].str.contains('organicdirect'))]

In [ ]:
Sense6Search['url'].str.split('search')[1]

In [ ]:
adwords['Campaign'].astype('str').str.split('_')[8963][1]

In [ ]:
USA['Campaign'].str.contains('sem_')[1]

In [ ]:
USA['Campaign'].astype('str').str.split('_')

In [ ]:
### Split into the SEM and non SEM
USA['Geo'] = ""
USA['Language'] = ""
USA['Brand'] = ""
USA['TargetingStrategy'] = ""
USA['Topic'] = ""
USA['Catchall'] = ""
for i in range(0,len(USA)):
    if USA['Campaign'].str.contains('sem_')[i]:
        USA['Geo'][i] = USA['Campaign'].astype('str').str.split('_')[i][1]
        USA['Language'][i] = USA['Campaign'].astype('str').str.split('_')[i][2]
        USA['Brand'][i] = USA['Campaign'].astype('str').str.split('_')[i][3]
        if len(USA['Campaign'].astype('str').str.split('_')[i]) > 4:
            USA['TargetingStrategy'][i] = USA['Campaign'].astype('str').str.split('_')[i][4]
            if len(USA['Campaign'].astype('str').str.split('_')[i]) > 5:
                USA['Topic'][i] = USA['Campaign'].astype('str').str.split('_')[i][5]
                if len(USA['Campaign'].astype('str').str.split('_')[i]) > 6:
                    USA['Catchall'][i] = USA['Campaign'].astype('str').str.split('_')[i][6:]  
    else:
        USA['Geo'][i] = USA['Campaign'].astype('str').str.split('_')[i][1]
        USA['Language'][i] = USA['Campaign'].astype('str').str.split('_')[i][2]
        USA['Topic'][i] = USA['Campaign'].astype('str').str.split('_')[i][0]
        USA['TargetingStrategy'][i] = USA['Ad group'].astype('str').str.split('_')[i][1]

In [ ]:
USA.sample(5)

In [ ]:
USA['Geo']

In [ ]:
print("{:.2f}% of the rows have campaign in the url".format(len(sense6[sense6['url'].astype(str).str.contains('campaign')])/len(sense6)))

In [ ]:
sense6[sense6['url'].astype(str).str.contains('elqcampaign')]['url'].astype('str').str.split('=')[57]

In [ ]:
## Filter to only rows that have ELQCT 
EloquaData = sense6[sense6['url'].astype(str).str.contains('elqct')]
EloquaData = EloquaData.reset_index()
print("{:.2f}% of rows retained".format(100*len(EloquaData)/len(sense6)))

In [ ]:
## Import TAL List
SFDC = pd.read_csv('TAL SFDC List.csv')
SFDC['Website']=SFDC['Website'].str.lstrip('www.')
SFDC['CountrySite']

In [ ]:
## Merge the two together
TALList = EloquaData.merge(SFDC, left_on='domain',right_on='Website',how='inner')
print("{:.2f}% of rows retained".format(100*len(TALList)/len(EloquaData)))

In [ ]:
TALList['Eloqua']=TALList['url']
i=0
for i in range(0,len(TALList)):
    TALList['Eloqua'][i]=TALList['url'][i].split('?')[1]


In [ ]:
TALList.to_csv('Merged TAL 6sense.csv')

In [ ]:
TALList['Eloqua'][101].split('&')

In [ ]:
TALList['6sense Account Buying Stage – Sailpoint'].value_counts()

In [ ]:
### Create a list of parameters
parameters=[]
z=[]
for i in range(0,len(TALList)):
    z=TALList['Eloqua'][i].split('&')
    for x in range(0,len(z)):
        if z[x].split('=')[0] in parameters:
            pass
        else:
            parameters.append(z[x].split('=')[0])

In [ ]:
parameters

In [ ]:
TALList['Category']=TALList['Eloqua']
TALList['Channel']=TALList['Eloqua']
i=0
for i in range(0,len(TALList)):
    TALList['Category'][i]=TALList['Eloqua'][i].split('&')[0]

In [ ]:
TALList['Eloqua'][].split('&')[0]

In [ ]:
TALList['Eloqua'][3].split('&')[1].split('=')[0]